# Hands-On Lab: Evaluating and Monitoring LLM Performance

---

## Scenario

You are an **LLM operations engineer** responsible for managing a production question-answering system used by internal employees across your organization. The system relies on a large language model deployed through **Databricks Model Serving**. In recent weeks, users have reported:

- **Inconsistent answer quality**
- **Occasional latency spikes**
- **Rising usage costs**

Leadership has asked you to diagnose these issues, evaluate system performance, and implement improvements using Databricks monitoring and evaluation tools.

### Your Workflow in This Lab

1. **Running structured evaluations** using MLflow to measure latency, token usage, and groundedness
2. **Querying inference tables** to identify patterns in real-world traffic, including token growth and slow prompts
3. **Using monitoring dashboards and agent traces** to investigate multi-step workflows
4. **Detecting anomalies and configuring alert conditions** for early issue detection
5. **Applying cost-optimization techniques** such as prompt refinement and context control
6. **Validating improvements** with repeatable evaluation runs

This lab mirrors real-world LLM observability and optimization challenges, where reliable performance and controlled operational cost are essential to business continuity. It also applies the best practices introduced throughout Chapter 8, including evaluation-dataset versioning, baseline-driven dashboard analysis, calibrated alerting strategies, and recurring review cycles that help maintain long-term system health. By completing this lab, you will integrate evaluation, monitoring, and optimization workflows to achieve end-to-end operational excellence.

---

## Learning Objectives

By the end of this lab, you will be able to:

1. ✅ **Design an end-to-end evaluation and monitoring workflow** for a production LLM system on Databricks
2. ✅ **Compare multiple model or prompt configurations** using MLflow evaluation metrics and justify a deployment choice
3. ✅ **Analyze inference tables** to diagnose latency, token usage, and RAG grounding issues
4. ✅ **Distinguish retrieval failures from model or prompt failures** using inference logs
5. ✅ **Use agent monitoring** to analyze multi-step workflows and identify inefficiencies
6. ✅ **Apply Databricks-native cost controls** and validate cost-saving changes without degrading output quality
7. ✅ **Select appropriate evaluation judges** and determine when ground truth is required

---

## Lab Duration

**Estimated Time:** 90-120 minutes

---

# Part 1: Prerequisites and Environment Setup

In this section, we will:
1. Install required libraries
2. Configure connection to Databricks workspace
3. Set up MLflow tracking
4. Generate synthetic Q&A data for our evaluation scenarios

---

### Step 1.1: Install Required Libraries

We need to install the necessary Python packages for working with Databricks, MLflow, and LLM evaluation. These packages provide the foundation for our monitoring and evaluation workflow.

In [0]:
# Install required packages for the lab
# - databricks-sdk: Official Databricks SDK for Python
# - mlflow[databricks]: MLflow with Databricks integration (includes GenAI evaluation)
# - openai: OpenAI client for model serving interactions
#
# Note: As of MLflow 3, the mlflow.genai namespace replaces databricks-agents
# for evaluation functionality. We install mlflow[databricks] for full integration.

%pip install databricks-sdk "mlflow[databricks]>=3.1.0" openai pandas numpy matplotlib seaborn --quiet

# Restart Python to pick up new packages (required in Databricks notebooks)
dbutils.library.restartPython()

### Step 1.2: Configure Databricks Connection

We'll set up the connection to our Databricks workspace using the SDK. This configuration enables us to:
- Create and manage Model Serving endpoints
- Access inference tables
- Configure monitoring and alerts

---

#### 📋 How to Find Your Workspace URL and ID

1. **Workspace URL**: Look at your browser's address bar when logged into Databricks. It follows this format:
   ```
   https://adb-<workspace-id>.<region>.azuredatabricks.net
   ```
   For example: `https://adb-YOUR-WORKSPACE-ID.azuredatabricks.net`

2. **Workspace ID**: The numeric value after `adb-` in your URL (e.g., `3141834805281315`)

3. **Alternative Method**:
   - Click on your **username** in the top-right corner
   - Select **User Settings**
   - The Workspace ID is displayed under **Workspace Info**

---

#### 🔑 How to Generate a Personal Access Token (PAT)

1. **Navigate to User Settings**:
   - Click on your **username** in the top-right corner of the Databricks workspace
   - Select **User Settings** from the dropdown menu

2. **Access Developer Settings**:
   - In the left sidebar, click on **Developer**
   - You'll see the **Access tokens** section

3. **Generate New Token**:
   - Click the **Manage** button next to Access tokens
   - Click **Generate new token**
   - Enter a **Comment** (e.g., "Chapter 8 Lab Token")
   - Set **Lifetime (days)** - leave blank for no expiration, or set a value like `90`
   - Click **Generate**

4. **Copy and Save the Token**:
   - ⚠️ **IMPORTANT**: Copy the token immediately! It will only be shown once.
   - The token starts with `dapi` (e.g., `dapixxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx-2`)
   - Store it securely - you'll need it for the configuration below

---

**🔒 Security Note:** In a production environment, store credentials securely using Databricks Secrets instead of hardcoding tokens in notebooks.

In [0]:
# Import required libraries for Databricks connection
import os
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput

# =============================================================================
# DATABRICKS WORKSPACE CONFIGURATION
# =============================================================================
# These credentials connect us to the Azure Databricks workspace
# In production, use Databricks Secrets instead of hardcoding tokens

DATABRICKS_HOST = "https://adb-YOUR-WORKSPACE-ID.azuredatabricks.net"
DATABRICKS_TOKEN = "dapi_YOUR_DATABRICKS_TOKEN_HERE-2"
CLUSTER_ID = "1130-045701-3mvg1aql"

# Set environment variables for SDK and MLflow
os.environ["DATABRICKS_HOST"] = DATABRICKS_HOST
os.environ["DATABRICKS_TOKEN"] = DATABRICKS_TOKEN

# Initialize the Workspace Client
# This client provides access to all Databricks APIs
w = WorkspaceClient(
    host=DATABRICKS_HOST,
    token=DATABRICKS_TOKEN
)

# Verify connection by listing current user
current_user = w.current_user.me()
print(f"✅ Connected to Databricks as: {current_user.user_name}")
print(f"📍 Workspace: {DATABRICKS_HOST}")

### Step 1.3: Configure MLflow Tracking

MLflow is our primary tool for tracking experiments, logging metrics, and evaluating LLM performance. We'll configure it to use the Databricks-hosted MLflow tracking server, which provides:

- **Centralized experiment tracking** across team members
- **Model versioning** through the Model Registry
- **Built-in LLM evaluation metrics** for quality assessment

In [0]:
import mlflow
from mlflow.tracking import MlflowClient

# =============================================================================
# MLFLOW CONFIGURATION
# =============================================================================
# Configure MLflow to use Databricks as the tracking server
# This enables centralized experiment management and model registry

mlflow.set_tracking_uri("databricks")

# Create an experiment for our LLM evaluation work
# Experiments group related runs together for easy comparison
EXPERIMENT_NAME = "/Users/" + current_user.user_name + "/llm_evaluation_monitoring_lab"

# Create or get the experiment
try:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
    print(f"✅ Created new experiment: {EXPERIMENT_NAME}")
except mlflow.exceptions.MlflowException:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    experiment_id = experiment.experiment_id
    print(f"✅ Using existing experiment: {EXPERIMENT_NAME}")

# Set the active experiment
mlflow.set_experiment(EXPERIMENT_NAME)
print(f"📊 Experiment ID: {experiment_id}")

### Step 1.4: Generate Synthetic Q&A Dataset

To simulate our production question-answering system, we'll create a synthetic dataset that includes:

- **Questions** of varying complexity (simple, moderate, complex)
- **Ground truth answers** for evaluation
- **Context documents** to test groundedness
- **Metadata** for categorization and analysis

This dataset will be used throughout the lab to:
1. Test our Model Serving endpoint
2. Evaluate response quality
3. Measure latency and token usage
4. Simulate production traffic patterns

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import json

# =============================================================================
# SYNTHETIC DATA GENERATION
# =============================================================================
# We create realistic Q&A data that mimics an internal knowledge base system
# This includes various question types, complexities, and domains

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Define question categories and their characteristics
CATEGORIES = {
    "hr_policy": {
        "questions": [
            "What is the company's remote work policy?",
            "How many vacation days do employees get per year?",
            "What is the process for requesting parental leave?",
            "How do I submit an expense report?",
            "What are the health insurance options available?"
        ],
        "contexts": [
            "The company allows employees to work remotely up to 3 days per week. Remote work requests must be approved by the direct manager. Employees must maintain core hours of 10 AM to 3 PM in their local timezone.",
            "Full-time employees receive 20 vacation days per year, accrued monthly. Unused vacation days can be carried over up to a maximum of 5 days. New employees are eligible for vacation after 90 days.",
            "Parental leave policy provides 16 weeks of paid leave for primary caregivers and 8 weeks for secondary caregivers. Leave must be requested at least 30 days in advance through the HR portal.",
            "Expense reports must be submitted within 30 days of the expense date. Use the Concur system to upload receipts and categorize expenses. Manager approval is required for expenses over $100.",
            "The company offers three health insurance tiers: Basic, Standard, and Premium. All plans include dental and vision coverage. Open enrollment occurs annually in November."
        ],
        "complexity": "simple"
    },
    "technical_docs": {
        "questions": [
            "How do I configure the API authentication for our microservices?",
            "What is the recommended approach for database connection pooling?",
            "How should I handle error logging in production applications?",
            "What are the security requirements for storing customer data?",
            "How do I set up CI/CD pipelines for new projects?"
        ],
        "contexts": [
            "API authentication uses OAuth 2.0 with JWT tokens. Services must register with the central auth server and obtain client credentials. Token expiration is set to 1 hour with refresh token support.",
            "Database connection pooling should use HikariCP with a minimum pool size of 5 and maximum of 20 connections. Connection timeout is set to 30 seconds. Enable connection validation on borrow.",
            "Production logging must use structured JSON format with correlation IDs. Log levels: ERROR for exceptions, WARN for recoverable issues, INFO for business events. Use ELK stack for aggregation.",
            "Customer data must be encrypted at rest using AES-256 and in transit using TLS 1.3. PII requires additional masking in logs. Data retention policy is 7 years for financial data.",
            "CI/CD pipelines use GitHub Actions with standardized templates. All projects must include unit tests (80% coverage), security scanning, and automated deployment to staging before production."
        ],
        "complexity": "moderate"
    },
    "strategic_planning": {
        "questions": [
            "What is our company's five-year growth strategy and how does it align with market trends?",
            "How should we approach the integration of AI capabilities across all product lines?",
            "What are the key risk factors in our current market expansion plan?",
            "How do we balance innovation investment with maintaining core business profitability?",
            "What organizational changes are needed to support our digital transformation initiative?"
        ],
        "contexts": [
            "The five-year strategy focuses on three pillars: market expansion into APAC region, product diversification through AI integration, and operational efficiency through automation. Target growth is 25% CAGR.",
            "AI integration roadmap includes: Phase 1 - customer service automation, Phase 2 - predictive analytics for sales, Phase 3 - AI-powered product features. Budget allocation is $50M over 3 years.",
            "Key risks include: regulatory changes in target markets, currency fluctuation exposure, talent acquisition challenges, and competitive pressure from well-funded startups. Mitigation strategies are documented.",
            "Innovation budget is set at 15% of revenue with quarterly review cycles. Core business must maintain 20% profit margin. New ventures have 18-month runway to demonstrate product-market fit.",
            "Digital transformation requires: flattening organizational hierarchy, creating cross-functional pods, investing in employee upskilling, and establishing a dedicated transformation office with C-suite sponsorship."
        ],
        "complexity": "complex"
    }
}

print("✅ Question categories defined")
print(f"📋 Categories: {list(CATEGORIES.keys())}")

#### Creating the Evaluation Dataset

**What we're doing:** Building a structured evaluation dataset that pairs each question with its "ground truth" (correct) answer. This is essential for measuring how well the LLM performs.

**How it works:**
1. **`generate_ground_truth_answer()` function**: Extracts the expected answer from the context based on complexity:
   - *Simple questions*: First sentence of context is the answer
   - *Moderate questions*: First two sentences combined
   - *Complex questions*: Full context needed for complete answer

2. **Dataset structure**: Each record contains:
   - `id`: Unique identifier (e.g., "product_info_1")
   - `question`: The user's question
   - `context`: Background information the LLM should use
   - `ground_truth`: The correct answer we expect
   - `category` and `complexity`: For stratified analysis
   - `expected_tokens`: Estimated token count for cost prediction

**Why this matters:** Without ground truth answers, we can't objectively measure if the LLM is giving correct responses. This dataset becomes our "answer key" for evaluation.

In [0]:
# =============================================================================
# CREATE EVALUATION DATASET
# =============================================================================
# Build a comprehensive dataset for LLM evaluation with ground truth answers

def generate_ground_truth_answer(question, context, complexity):
    """
    Generate a ground truth answer based on the context.
    In a real scenario, these would be human-curated answers.
    """
    # Extract key information from context as the ground truth
    sentences = context.split(". ")
    if complexity == "simple":
        return sentences[0] + "." if sentences else context
    elif complexity == "moderate":
        return ". ".join(sentences[:2]) + "." if len(sentences) >= 2 else context
    else:  # complex
        return context

# Build the evaluation dataset
eval_data = []

for category, data in CATEGORIES.items():
    for i, (question, context) in enumerate(zip(data["questions"], data["contexts"])):
        ground_truth = generate_ground_truth_answer(question, context, data["complexity"])

        eval_data.append({
            "id": f"{category}_{i+1}",
            "question": question,
            "context": context,
            "ground_truth": ground_truth,
            "category": category,
            "complexity": data["complexity"],
            "expected_tokens": len(context.split()) * 2  # Rough estimate
        })

# Create DataFrame
eval_df = pd.DataFrame(eval_data)

print(f"✅ Created evaluation dataset with {len(eval_df)} samples")
print(f"\n📊 Dataset Summary:")
print(eval_df.groupby(["category", "complexity"]).size().unstack(fill_value=0))
display(eval_df.head())

#### Generating Simulated Production Traffic

**What we're doing:** Creating synthetic inference logs that mimic what a real production LLM endpoint would generate over 7 days of operation.

**How the `generate_traffic_data()` function works:**

1. **Time distribution**: Generates timestamps over 7 days with realistic patterns:
   - More requests during business hours (9 AM - 6 PM)
   - Fewer requests on weekends and nights

2. **Latency simulation** using normal distribution:
   ```python
   base_latency = {"simple": 200, "moderate": 400, "complex": 800}
   latency = np.random.normal(base_latency, std_dev)
   ```
   - Simple queries: ~200ms average
   - Complex queries: ~800ms average
   - 5% of requests get artificial "spikes" (3-5x normal latency)

3. **Token usage**: Calculated based on complexity with random variation

4. **Error injection**: 2% of requests marked as "error" or "timeout" to simulate real failures

**Why this matters:** Real production data takes weeks to accumulate. Synthetic data lets us immediately practice monitoring, anomaly detection, and alerting without waiting.

In [0]:
# =============================================================================
# GENERATE SIMULATED PRODUCTION TRAFFIC DATA
# =============================================================================
# Create synthetic inference logs to simulate production traffic patterns
# This data will be used for monitoring and anomaly detection exercises

def generate_traffic_data(num_records=500):
    """
    Generate synthetic production traffic data with realistic patterns:
    - Normal latency with occasional spikes
    - Varying token usage based on question complexity
    - Time-based patterns (higher traffic during business hours)
    """
    traffic_data = []
    base_time = datetime.now() - timedelta(days=7)

    for i in range(num_records):
        # Select random question from our dataset
        sample = random.choice(eval_data)

        # Generate timestamp with business hour bias
        hour_offset = random.gauss(14, 4)  # Peak around 2 PM
        hour_offset = max(8, min(20, hour_offset))  # Clamp to business hours
        timestamp = base_time + timedelta(
            days=random.randint(0, 6),
            hours=hour_offset,
            minutes=random.randint(0, 59)
        )

        # Generate latency based on complexity with occasional spikes
        base_latency = {"simple": 200, "moderate": 400, "complex": 800}
        latency = base_latency[sample["complexity"]]
        latency += random.gauss(0, latency * 0.2)  # Add noise

        # Inject latency spikes (5% of requests)
        if random.random() < 0.05:
            latency *= random.uniform(3, 8)  # Spike multiplier

        # Generate token counts
        input_tokens = len(sample["question"].split()) + len(sample["context"].split())
        output_tokens = int(sample["expected_tokens"] * random.uniform(0.8, 1.2))

        # Inject token anomalies (3% of requests)
        if random.random() < 0.03:
            output_tokens *= random.randint(3, 5)  # Unexpectedly long responses

        # Generate quality score (simulated)
        base_quality = {"simple": 0.9, "moderate": 0.8, "complex": 0.7}
        quality_score = base_quality[sample["complexity"]] + random.gauss(0, 0.1)
        quality_score = max(0, min(1, quality_score))  # Clamp to [0, 1]

        traffic_data.append({
            "request_id": f"req_{i:06d}",
            "timestamp": timestamp,
            "question": sample["question"],
            "category": sample["category"],
            "complexity": sample["complexity"],
            "latency_ms": round(latency, 2),
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "total_tokens": input_tokens + output_tokens,
            "quality_score": round(quality_score, 3),
            "status": "success" if random.random() > 0.02 else "error"
        })

    return pd.DataFrame(traffic_data)

# Generate traffic data
traffic_df = generate_traffic_data(500)

print(f"✅ Generated {len(traffic_df)} synthetic traffic records")
print(f"\n📊 Traffic Summary:")
print(f"   Date Range: {traffic_df['timestamp'].min()} to {traffic_df['timestamp'].max()}")
print(f"   Avg Latency: {traffic_df['latency_ms'].mean():.2f} ms")
print(f"   Avg Tokens: {traffic_df['total_tokens'].mean():.1f}")
print(f"   Error Rate: {(traffic_df['status'] == 'error').mean()*100:.1f}%")
display(traffic_df.head())

#### Saving Datasets to Delta Tables

**What we're doing:** Persisting our synthetic datasets to Delta Lake tables in the Unity Catalog so they survive cluster restarts and can be queried by other notebooks.

**How it works:**

1. **Create schema**: `spark.sql("CREATE SCHEMA IF NOT EXISTS main.llm_monitoring_lab")`
   - Creates a namespace to organize our tables

2. **Convert and save**:
   ```python
   spark.createDataFrame(pandas_df)  # Convert pandas → Spark DataFrame
   .write.mode("overwrite")          # Replace if exists
   .saveAsTable("catalog.schema.table")  # Save as managed Delta table
   ```

3. **Tables created**:
   - `evaluation_dataset`: Our Q&A pairs with ground truth answers
   - `traffic_data`: Synthetic production traffic logs

**Why Delta Lake?**
- **ACID transactions**: Safe concurrent reads/writes
- **Time travel**: Query previous versions with `VERSION AS OF`
- **Schema enforcement**: Prevents bad data from corrupting tables
- **Optimized queries**: Auto-compaction and Z-ordering for fast reads

In [0]:
# =============================================================================
# SAVE DATASETS TO DATABRICKS
# =============================================================================
# Store our synthetic data in Delta tables for use throughout the lab

# Define catalog and schema (using default catalog)
CATALOG = "main"
SCHEMA = "llm_monitoring_lab"

# Create schema if it doesn't exist
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")

# Convert pandas DataFrames to Spark DataFrames and save as Delta tables
eval_spark_df = spark.createDataFrame(eval_df)
eval_spark_df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.evaluation_dataset")

traffic_spark_df = spark.createDataFrame(traffic_df)
traffic_spark_df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.traffic_data")

print(f"✅ Saved evaluation dataset to {CATALOG}.{SCHEMA}.evaluation_dataset")
print(f"✅ Saved traffic data to {CATALOG}.{SCHEMA}.traffic_data")

---

# Part 2: Creating and Deploying the Model Serving Endpoint

In this section, we will:
1. Register a foundation model for serving
2. Create a Model Serving endpoint
3. Configure inference tables for logging
4. Test the endpoint with sample queries

This endpoint will serve as our production Q&A system that we'll monitor and optimize throughout the lab.

---

### Step 2.1: Configure Foundation Model API Access

Databricks provides **Foundation Model APIs** - pre-deployed, pay-per-token endpoints for popular LLMs. These are:

- **Immediately available** - no deployment required
- **Auto-scaling** - handles any traffic volume
- **Pay-per-token** - cost-effective for variable workloads
- **OpenAI-compatible** - easy integration with existing code

We'll use the **Meta Llama 3.1 8B Instruct** model (`databricks-meta-llama-3-1-8b-instruct`), which is well-suited for Q&A tasks.

For inference table logging, we'll create a **custom proxy endpoint** that wraps the foundation model and enables request/response logging.

In [0]:
import time

# =============================================================================
# MODEL SERVING ENDPOINT CONFIGURATION
# =============================================================================
# Databricks Foundation Model APIs are pre-deployed and ready to use
# No custom endpoint creation needed - we query them directly

FOUNDATION_MODEL = "databricks-meta-llama-3-1-8b-instruct"

# First, let's verify the Foundation Model is available
print("🔍 Checking available Foundation Model APIs...")

# List foundation model endpoints
try:
    all_endpoints = list(w.serving_endpoints.list())
    foundation_endpoints = [e for e in all_endpoints if e.name.startswith("databricks-")]
    print(f"   Found {len(foundation_endpoints)} Foundation Model endpoints")

    # Check if our model is available
    available_models = [e.name for e in foundation_endpoints]
    if FOUNDATION_MODEL in available_models:
        print(f"   ✅ {FOUNDATION_MODEL} is available")
    else:
        print(f"   ⚠️ {FOUNDATION_MODEL} not found in list. Available models:")
        for model in available_models[:5]:
            print(f"      - {model}")
        print(f"   Note: The model may still work - Foundation Models are always available")
except Exception as e:
    print(f"   Note: Could not list endpoints ({e})")
    print(f"   Proceeding with Foundation Model - these are always available")

# For this lab, we'll use the Foundation Model directly
# The endpoint name for queries will be the foundation model name
ENDPOINT_NAME = FOUNDATION_MODEL

print(f"\n✅ Using Foundation Model: {ENDPOINT_NAME}")
print(f"   Endpoint URL: {DATABRICKS_HOST}/serving-endpoints/{ENDPOINT_NAME}/invocations")

#### Verifying the Foundation Model Endpoint

**What we're doing:** Confirming that the Databricks Foundation Model API is accessible before we start sending queries.

**How it works:**

1. **SDK endpoint check**: `w.serving_endpoints.get(name=ENDPOINT_NAME)`
   - Retrieves metadata about the endpoint
   - Confirms the endpoint exists and is in "READY" state

2. **What we verify**:
   - Endpoint name matches our configuration
   - State is "READY" (not "PENDING" or "FAILED")
   - URL is correctly formed for API calls

**Foundation Models vs Custom Endpoints:**

| Aspect | Foundation Model | Custom Endpoint |
|--------|-----------------|-----------------|
| Provisioning | Always ready | Minutes to hours |
| Scaling | Automatic | Manual configuration |
| Billing | Pay-per-token | Pay for compute time |
| Customization | None | Fine-tuning possible |

**Why this matters:** Verifying the endpoint upfront prevents confusing errors later. If the endpoint isn't accessible, we want to know immediately rather than after running expensive evaluation jobs.

In [0]:
# =============================================================================
# VERIFY FOUNDATION MODEL ENDPOINT
# =============================================================================
# Foundation Model APIs are always ready - no provisioning needed
# Let's verify the endpoint is accessible

try:
    endpoint_info = w.serving_endpoints.get(name=ENDPOINT_NAME)
    print(f"✅ Foundation Model Endpoint Verified")
    print(f"\n📊 Endpoint Details:")
    print(f"   Name: {endpoint_info.name}")
    print(f"   State: {endpoint_info.state.ready}")
    print(f"   URL: {DATABRICKS_HOST}/serving-endpoints/{ENDPOINT_NAME}/invocations")

    # Check endpoint configuration
    if endpoint_info.config and endpoint_info.config.served_entities:
        for entity in endpoint_info.config.served_entities:
            print(f"\n   Served Entity:")
            print(f"      Name: {entity.entity_name if hasattr(entity, 'entity_name') else 'N/A'}")
except Exception as e:
    print(f"⚠️ Could not get endpoint details: {str(e)}")
    print(f"   This is expected for Foundation Model APIs - they work without explicit configuration")
    print(f"\n✅ Proceeding with endpoint: {ENDPOINT_NAME}")

### Step 2.2: Test the Model Serving Endpoint

Now we'll test our endpoint with sample queries to verify it's working correctly. We'll use the OpenAI-compatible API that Databricks provides for foundation models.

In [0]:
from openai import OpenAI

# =============================================================================
# TEST MODEL SERVING ENDPOINT
# =============================================================================
# Use OpenAI-compatible client to query our endpoint

# Disable MLflow's automatic tracing for OpenAI calls to avoid duplicate trace ID warnings
# This happens because MLflow tries to auto-trace LLM calls, but re-running cells
# can cause trace ID conflicts
try:
    mlflow.openai.autolog(disable=True)
except AttributeError:
    # Older MLflow versions may not have this - that's OK
    pass

# Initialize OpenAI client pointing to Databricks
client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url=f"{DATABRICKS_HOST}/serving-endpoints"
)

def query_qa_endpoint(question, context, max_tokens=500):
    """
    Query the Q&A endpoint with a question and context.
    Returns the response and timing information.
    """
    # Construct the prompt for Q&A
    system_prompt = """You are a helpful assistant that answers questions based on the provided context.
    Be concise and accurate. If the answer is not in the context, say so."""

    user_prompt = f"""Context: {context}

Question: {question}

Answer:"""

    start_time = time.time()

    response = client.chat.completions.create(
        model=ENDPOINT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=max_tokens,
        temperature=0.1  # Low temperature for factual responses
    )

    latency_ms = (time.time() - start_time) * 1000

    return {
        "answer": response.choices[0].message.content,
        "latency_ms": latency_ms,
        "input_tokens": response.usage.prompt_tokens,
        "output_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens
    }

# Test with a sample question
test_sample = eval_data[0]
print(f"📝 Testing with question: {test_sample['question']}")
print(f"📄 Context: {test_sample['context'][:100]}...")

result = query_qa_endpoint(test_sample["question"], test_sample["context"])

print(f"\n✅ Response received:")
print(f"   Answer: {result['answer']}")
print(f"   Latency: {result['latency_ms']:.2f} ms")
print(f"   Tokens: {result['total_tokens']} (input: {result['input_tokens']}, output: {result['output_tokens']})")

#### Establishing Baseline Performance Metrics

**What we're doing:** Running multiple queries to collect baseline performance data that will inform our monitoring thresholds.

**How it works:**

1. **Batch execution loop**:
   ```python
   for sample in eval_data[:10]:
       result = query_qa_endpoint(sample["question"], sample["context"])
       batch_results.append(result)
       time.sleep(1)  # Rate limiting to avoid throttling
   ```

2. **Metrics collected per query**:
   - `latency_ms`: End-to-end response time
   - `input_tokens`: Tokens in prompt (question + context)
   - `output_tokens`: Tokens in generated answer
   - `category` and `complexity`: For stratified analysis

3. **Baseline statistics calculated**:
   - Mean, P50 (median), P95, P99 latency
   - Average token usage by complexity
   - Success rate

**Why baselines matter (Chapter 8 best practice):**
- **Threshold setting**: "Alert when latency > P95 baseline × 1.5"
- **Drift detection**: Compare current metrics to baseline
- **Capacity planning**: Predict costs based on token patterns
- **SLA definition**: Set realistic performance guarantees

Without baselines, you're guessing at what "normal" looks like.

In [0]:
# =============================================================================
# BATCH TEST FOR BASELINE METRICS
# =============================================================================
# Run multiple queries to establish baseline performance metrics

print("🔄 Running batch test to establish baseline metrics...")
print("   This will take a few minutes...\n")

batch_results = []

# Test with a subset of our evaluation data
for i, sample in enumerate(eval_data[:10]):
    try:
        result = query_qa_endpoint(sample["question"], sample["context"])
        result["id"] = sample["id"]
        result["category"] = sample["category"]
        result["complexity"] = sample["complexity"]
        result["ground_truth"] = sample["ground_truth"]
        batch_results.append(result)
        print(f"   ✓ Completed {i+1}/10: {sample['id']} ({result['latency_ms']:.0f}ms)")
    except Exception as e:
        print(f"   ✗ Failed {i+1}/10: {sample['id']} - {str(e)}")

    # Small delay to avoid rate limiting
    time.sleep(1)

# Create results DataFrame
batch_df = pd.DataFrame(batch_results)

print(f"\n📊 Baseline Performance Summary:")
print(f"   Average Latency: {batch_df['latency_ms'].mean():.2f} ms")
print(f"   P95 Latency: {batch_df['latency_ms'].quantile(0.95):.2f} ms")
print(f"   Average Tokens: {batch_df['total_tokens'].mean():.1f}")
print(f"\n   By Complexity:")
print(batch_df.groupby('complexity')[['latency_ms', 'total_tokens']].mean())

---

# Part 3: Structured Evaluation with MLflow

In this section, we will:
1. Set up MLflow evaluation for LLM responses
2. Measure quality metrics (groundedness, relevance, coherence)
3. Track latency and token usage metrics
4. Compare baseline vs. optimized performance

MLflow provides built-in LLM evaluation capabilities that help us systematically assess model quality.

---

### Step 3.1: Configure MLflow LLM Evaluation

**What we're doing:** Setting up custom evaluation functions using the LLM-as-Judge pattern.

**How our evaluation works:**

1. **Custom evaluation functions**: We create functions that call an LLM to judge responses
   ```python
   def evaluate_groundedness(question: str, context: str, answer: str) -> int:
       # Use LLM-as-judge to score the output
       return score  # 1-5
   ```

2. **Function parameters**: Each function receives explicit parameters:
   - `question`: The user's question
   - `context`: The retrieved context (for groundedness)
   - `answer`: The model's generated response
   - `ground_truth`: Expected answer (for correctness)

3. **LLM-as-Judge pattern**: We use Llama 8B to evaluate responses

4. **MLflow 3 compatibility**: We also provide wrapper functions (`groundedness_scorer`, `correctness_scorer`) that follow the MLflow 3 `@scorer` signature for use with `mlflow.genai.evaluate()` if available.

**Why this approach?**
- Clear, explicit function signatures
- Easy to understand and debug
- Compatible with both manual evaluation and MLflow 3 GenAI API

In [0]:
# =============================================================================
# MLFLOW GENAI EVALUATION SETUP (New API - MLflow 3+)
# =============================================================================
# Configure evaluation scorers for our Q&A system
#
# MLflow 3 introduces the @scorer decorator with standardized signature:
#   def my_scorer(inputs, outputs, expectations=None, traces=None)
#
# For compatibility, we'll create simple scoring functions that can be used
# both with the new API and called directly for manual evaluation.

# Use the same foundation model for judging (8B model available in workspace)
JUDGE_MODEL = FOUNDATION_MODEL  # databricks-meta-llama-3-1-8b-instruct

def evaluate_groundedness(question: str, context: str, answer: str) -> int:
    """
    Evaluate if the answer is grounded in the provided context.
    Returns a score from 1-5.
    """
    from openai import OpenAI

    judge_client = OpenAI(
        api_key=DATABRICKS_TOKEN,
        base_url=f"{DATABRICKS_HOST}/serving-endpoints"
    )

    prompt = f"""You are evaluating whether an answer is grounded in the provided context.

Context: {context}
Question: {question}
Answer: {answer}

Score the groundedness from 1-5:
1 = Answer contains claims not supported by context
2 = Answer partially supported but has unsupported claims
3 = Answer mostly supported with minor gaps
4 = Answer well supported by context
5 = Answer fully grounded in context

Respond with ONLY a single number (1-5)."""

    try:
        response = judge_client.chat.completions.create(
            model=JUDGE_MODEL,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=10,
            temperature=0
        )
        score = int(response.choices[0].message.content.strip()[0])
        return max(1, min(5, score))  # Ensure score is between 1-5
    except Exception as e:
        print(f"   Warning: Groundedness evaluation failed: {e}")
        return 3  # Default to middle score if evaluation fails

def evaluate_correctness(question: str, ground_truth: str, answer: str) -> int:
    """
    Evaluate if the answer correctly addresses the question based on ground truth.
    Returns a score from 1-5.
    """
    from openai import OpenAI

    judge_client = OpenAI(
        api_key=DATABRICKS_TOKEN,
        base_url=f"{DATABRICKS_HOST}/serving-endpoints"
    )

    prompt = f"""Compare the generated answer with the ground truth answer.

Question: {question}
Ground Truth: {ground_truth}
Generated Answer: {answer}

Score from 1-5:
1 = Completely incorrect or contradicts ground truth
2 = Partially correct but missing key information
3 = Mostly correct with some inaccuracies
4 = Correct with minor differences in wording
5 = Fully correct and complete

Respond with ONLY a single number (1-5)."""

    try:
        response = judge_client.chat.completions.create(
            model=JUDGE_MODEL,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=10,
            temperature=0
        )
        score = int(response.choices[0].message.content.strip()[0])
        return max(1, min(5, score))  # Ensure score is between 1-5
    except Exception as e:
        print(f"   Warning: Correctness evaluation failed: {e}")
        return 3  # Default to middle score if evaluation fails

# Wrapper functions compatible with MLflow 3 @scorer signature
# These can be used with mlflow.genai.evaluate() if available
def groundedness_scorer(inputs, outputs, expectations=None, traces=None):
    """MLflow 3 compatible scorer wrapper for groundedness."""
    question = inputs.get('question', '') if isinstance(inputs, dict) else str(inputs)
    context = inputs.get('context', '') if isinstance(inputs, dict) else ''
    answer = outputs.get('response', outputs) if isinstance(outputs, dict) else str(outputs)
    return evaluate_groundedness(question, context, answer)

def correctness_scorer(inputs, outputs, expectations=None, traces=None):
    """MLflow 3 compatible scorer wrapper for correctness."""
    question = inputs.get('question', '') if isinstance(inputs, dict) else str(inputs)
    ground_truth = expectations.get('ground_truth', '') if expectations else ''
    answer = outputs.get('response', outputs) if isinstance(outputs, dict) else str(outputs)
    return evaluate_correctness(question, ground_truth, answer)

print("✅ Custom evaluation scorers configured:")
print(f"   - Judge model: {JUDGE_MODEL}")
print("   - evaluate_groundedness(): Measures factual support from context")
print("   - evaluate_correctness(): Compares with ground truth answers")
print("   - MLflow 3 compatible wrappers: groundedness_scorer, correctness_scorer")

### Step 3.2: Run Comprehensive Evaluation

Now we'll run a full evaluation using MLflow. This will:
1. Query our endpoint for each evaluation sample
2. Calculate quality metrics using LLM-as-judge
3. Log all results to MLflow for tracking
4. Generate a comprehensive evaluation report

In [0]:
# =============================================================================
# RUN MLFLOW EVALUATION
# =============================================================================
# Execute comprehensive evaluation and log results

# Prepare evaluation data with model outputs
eval_results = []

print("🔄 Running comprehensive evaluation...")
print("   Querying endpoint and collecting responses...\n")

for i, sample in enumerate(eval_data):
    try:
        # Query the endpoint
        result = query_qa_endpoint(sample["question"], sample["context"])

        eval_results.append({
            "id": sample["id"],
            "question": sample["question"],
            "context": sample["context"],
            "ground_truth": sample["ground_truth"],
            "output": result["answer"],  # MLflow expects 'output' column
            "category": sample["category"],
            "complexity": sample["complexity"],
            "latency_ms": result["latency_ms"],
            "input_tokens": result["input_tokens"],
            "output_tokens": result["output_tokens"],
            "total_tokens": result["total_tokens"]
        })

        print(f"   ✓ {i+1}/{len(eval_data)}: {sample['id']}")

    except Exception as e:
        print(f"   ✗ {i+1}/{len(eval_data)}: {sample['id']} - {str(e)}")

    time.sleep(0.5)  # Rate limiting

# Create evaluation DataFrame
eval_results_df = pd.DataFrame(eval_results)
print(f"\n✅ Collected {len(eval_results_df)} evaluation samples")

#### Logging Evaluation Results to MLflow

**What we're doing:** Running our custom scorers on each sample and logging results to MLflow.

**How the new GenAI API works:**

1. **Prepare data in new format**:
   ```python
   eval_data_for_genai.append({
       "inputs": {"question": ..., "context": ...},
       "outputs": row["output"],
       "expectations": {"ground_truth": ...}
   })
   ```

2. **Call evaluation functions directly**:
   ```python
   g_score = evaluate_groundedness(question=..., context=..., answer=...)
   c_score = evaluate_correctness(question=..., ground_truth=..., answer=...)
   ```
   - Each function calls the judge LLM (Llama 8B)
   - Returns a score from 1-5

3. **Log aggregate metrics**:
   ```python
   mlflow.log_metric("groundedness_mean", avg_groundedness)
   mlflow.log_metric("answer_correctness_mean", avg_correctness)
   ```

**Why this approach?** We use explicit function calls for clarity and control. The MLflow 3 compatible wrappers (`groundedness_scorer`, `correctness_scorer`) can be used with `mlflow.genai.evaluate()` if available.

In [0]:
# =============================================================================
# LOG EVALUATION TO MLFLOW
# =============================================================================
# Run MLflow evaluation with our custom scorers (New GenAI API)

with mlflow.start_run(run_name="baseline_evaluation") as run:

    # Log evaluation parameters
    mlflow.log_param("endpoint_name", ENDPOINT_NAME)
    mlflow.log_param("model_name", FOUNDATION_MODEL)
    mlflow.log_param("num_samples", len(eval_results_df))
    mlflow.log_param("evaluation_date", datetime.now().isoformat())

    # Prepare data in the new format for mlflow.genai.evaluate()
    # The new API expects: inputs (dict), outputs (str), expectations (dict)
    eval_data_for_genai = []
    for _, row in eval_results_df.iterrows():
        eval_data_for_genai.append({
            "inputs": {
                "question": row["question"],
                "context": row["context"]
            },
            "outputs": row["output"],
            "expectations": {
                "ground_truth": row["ground_truth"]
            }
        })

    # Run evaluation with custom scorers
    print("🔄 Running MLflow evaluation with LLM-as-judge scorers...")
    print("   (Using new MLflow 3.4+ GenAI API)")

    # Calculate scores manually and log them
    groundedness_scores = []
    correctness_scores = []

    for i, item in enumerate(eval_data_for_genai):
        try:
            # Use the evaluation functions with explicit parameters
            g_score = evaluate_groundedness(
                question=item["inputs"]["question"],
                context=item["inputs"]["context"],
                answer=item["outputs"]
            )
            c_score = evaluate_correctness(
                question=item["inputs"]["question"],
                ground_truth=item["expectations"]["ground_truth"],
                answer=item["outputs"]
            )
            groundedness_scores.append(g_score)
            correctness_scores.append(c_score)
            if (i + 1) % 5 == 0:
                print(f"   Evaluated {i + 1}/{len(eval_data_for_genai)} samples...")
        except Exception as e:
            print(f"   Warning: Error evaluating sample {i}: {e}")
            groundedness_scores.append(3)
            correctness_scores.append(3)
        time.sleep(0.5)  # Rate limiting for judge model

    # Calculate aggregate metrics
    avg_groundedness = sum(groundedness_scores) / len(groundedness_scores)
    avg_correctness = sum(correctness_scores) / len(correctness_scores)

    # Log LLM-as-judge metrics
    mlflow.log_metric("groundedness_mean", avg_groundedness)
    mlflow.log_metric("answer_correctness_mean", avg_correctness)

    # Log additional performance metrics
    mlflow.log_metric("avg_latency_ms", eval_results_df["latency_ms"].mean())
    mlflow.log_metric("p95_latency_ms", eval_results_df["latency_ms"].quantile(0.95))
    mlflow.log_metric("avg_total_tokens", eval_results_df["total_tokens"].mean())
    mlflow.log_metric("total_input_tokens", eval_results_df["input_tokens"].sum())
    mlflow.log_metric("total_output_tokens", eval_results_df["output_tokens"].sum())

    # Add scores to dataframe and save
    eval_results_df["groundedness_score"] = groundedness_scores
    eval_results_df["correctness_score"] = correctness_scores

    # Log the evaluation dataset as artifact
    eval_results_df.to_csv("/tmp/evaluation_results.csv", index=False)
    mlflow.log_artifact("/tmp/evaluation_results.csv")

    print(f"\n✅ Evaluation logged to MLflow")
    print(f"   Run ID: {run.info.run_id}")
    print(f"   Experiment: {EXPERIMENT_NAME}")

    # Display metrics summary
    print(f"\n📊 Evaluation Metrics Summary:")
    print(f"   groundedness_mean: {avg_groundedness:.2f}")
    print(f"   answer_correctness_mean: {avg_correctness:.2f}")
    print(f"   avg_latency_ms: {eval_results_df['latency_ms'].mean():.2f}")
    print(f"   p95_latency_ms: {eval_results_df['latency_ms'].quantile(0.95):.2f}")
    print(f"   avg_total_tokens: {eval_results_df['total_tokens'].mean():.1f}")

### Step 3.3: Selecting Evaluation Judges and Ground Truth Requirements

**Learning Objective 7:** Select appropriate evaluation judges and determine when ground truth is required.

When evaluating LLM outputs, you must choose the right evaluation approach. This decision depends on what you're measuring and what reference data you have available.

---

#### When to Use Different Evaluation Approaches

| Evaluation Type | When to Use | Ground Truth Required? | Example Metrics |
|-----------------|-------------|------------------------|-----------------|
| **LLM-as-Judge** | Subjective quality (coherence, helpfulness, tone) | No | Groundedness, Relevance, Fluency |
| **Ground Truth Comparison** | Factual correctness, exact answers | Yes | Answer Correctness, F1 Score |
| **Heuristic/Rule-Based** | Format validation, length checks | No | Token count, Response format |
| **Human Evaluation** | High-stakes decisions, edge cases | Optional | Expert ratings, A/B preferences |

---

#### LLM-as-Judge: Pros and Cons

**Advantages:**
- Scales to thousands of evaluations automatically
- Can assess subjective qualities (tone, helpfulness)
- No need to maintain ground truth datasets

**Limitations:**
- Judge model may have biases (prefers verbose answers)
- Smaller models (8B) less reliable than larger ones (70B+)
- Cannot verify factual accuracy without context

**Best Practice:** Use a larger model as judge than the model being evaluated when possible.

---

#### When Ground Truth is Required

Ground truth is **essential** when:
1. **Factual accuracy matters** - Medical, legal, financial domains
2. **Exact answers expected** - Math problems, code generation
3. **Compliance requirements** - Auditable evaluation trails
4. **Regression testing** - Comparing model versions on known cases

Ground truth is **optional** when:
1. **Subjective quality** - "Is this response helpful?"
2. **Style/tone evaluation** - "Is this professional?"
3. **Exploratory analysis** - Understanding failure modes

In [0]:
# =============================================================================
# DEMONSTRATION: CHOOSING EVALUATION APPROACH
# =============================================================================
# This cell demonstrates how to select the right evaluation approach

def select_evaluation_approach(use_case: str) -> dict:
    """
    Helper function to recommend evaluation approach based on use case.

    Returns recommended approach with rationale.
    """
    approaches = {
        "factual_qa": {
            "approach": "Ground Truth + LLM-as-Judge",
            "ground_truth_required": True,
            "recommended_metrics": ["answer_correctness", "groundedness"],
            "rationale": "Factual Q&A requires verifiable answers. Use ground truth for correctness, LLM-as-judge for groundedness."
        },
        "creative_writing": {
            "approach": "LLM-as-Judge Only",
            "ground_truth_required": False,
            "recommended_metrics": ["coherence", "creativity", "tone"],
            "rationale": "Creative tasks have no single correct answer. LLM judges can assess quality dimensions."
        },
        "code_generation": {
            "approach": "Execution-Based + Ground Truth",
            "ground_truth_required": True,
            "recommended_metrics": ["test_pass_rate", "syntax_validity"],
            "rationale": "Code can be objectively tested. Run tests and compare outputs."
        },
        "summarization": {
            "approach": "LLM-as-Judge + Optional Ground Truth",
            "ground_truth_required": False,
            "recommended_metrics": ["faithfulness", "conciseness", "coverage"],
            "rationale": "Summaries can vary. LLM judges assess quality; ground truth optional for key points."
        },
        "rag_qa": {
            "approach": "Ground Truth + Groundedness Check",
            "ground_truth_required": True,
            "recommended_metrics": ["answer_correctness", "groundedness", "context_relevance"],
            "rationale": "RAG systems need both correct answers AND grounding in retrieved context."
        }
    }

    return approaches.get(use_case, {
        "approach": "LLM-as-Judge",
        "ground_truth_required": False,
        "recommended_metrics": ["relevance", "coherence"],
        "rationale": "Default to LLM-as-judge for general evaluation."
    })

# Demonstrate for our Q&A use case
our_use_case = select_evaluation_approach("rag_qa")

print("📋 Evaluation Approach for Our RAG Q&A System:")
print(f"   Approach: {our_use_case['approach']}")
print(f"   Ground Truth Required: {our_use_case['ground_truth_required']}")
print(f"   Recommended Metrics: {', '.join(our_use_case['recommended_metrics'])}")
print(f"   Rationale: {our_use_case['rationale']}")

print("\n" + "="*60)
print("In this lab, we use:")
print("   ✓ Ground truth answers (from our evaluation dataset)")
print("   ✓ LLM-as-judge (groundedness scorer)")
print("   ✓ Answer correctness (compares to ground truth)")
print("This combination is ideal for RAG-based Q&A systems.")

---

# Part 4: Analyzing Inference Tables

In this section, we will:
1. Query inference tables to analyze production traffic
2. Identify latency patterns and anomalies
3. Analyze token usage trends
4. Detect problematic query patterns

Inference tables automatically capture all requests to Model Serving endpoints, providing rich data for monitoring and debugging.

---

### Step 4.1: Query Inference Tables

Databricks automatically logs all inference requests to Delta tables when auto-capture is enabled. These tables contain:

- **Request payloads**: Input prompts and parameters
- **Response data**: Model outputs and metadata
- **Timing information**: Latency measurements
- **Token counts**: Usage metrics for cost analysis

In [0]:
# =============================================================================
# QUERY INFERENCE TABLES
# =============================================================================
# Analyze production traffic patterns from inference logs

# For this lab, we'll use our synthetic traffic data
# In production, you would query the actual inference tables:
# inference_table = f"{CATALOG}.{SCHEMA}.qa_endpoint_payload"

# Load our synthetic traffic data
traffic_analysis_df = spark.table(f"{CATALOG}.{SCHEMA}.traffic_data").toPandas()

print(f"📊 Analyzing {len(traffic_analysis_df)} inference records")
print(f"   Time Range: {traffic_analysis_df['timestamp'].min()} to {traffic_analysis_df['timestamp'].max()}")

# Basic statistics
print(f"\n📈 Traffic Statistics:")
print(f"   Total Requests: {len(traffic_analysis_df)}")
print(f"   Success Rate: {(traffic_analysis_df['status'] == 'success').mean()*100:.1f}%")
print(f"   Avg Latency: {traffic_analysis_df['latency_ms'].mean():.2f} ms")
print(f"   P50 Latency: {traffic_analysis_df['latency_ms'].median():.2f} ms")
print(f"   P95 Latency: {traffic_analysis_df['latency_ms'].quantile(0.95):.2f} ms")
print(f"   P99 Latency: {traffic_analysis_df['latency_ms'].quantile(0.99):.2f} ms")

### Step 4.2: Identify Latency Patterns

We'll analyze latency patterns to identify:
- Time-based variations (peak hours vs. off-peak)
- Complexity-based differences
- Anomalous slow requests

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

# =============================================================================
# LATENCY PATTERN ANALYSIS
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Latency distribution by complexity
ax1 = axes[0, 0]
for complexity in ['simple', 'moderate', 'complex']:
    data = traffic_analysis_df[traffic_analysis_df['complexity'] == complexity]['latency_ms']
    ax1.hist(data, bins=30, alpha=0.5, label=complexity)
ax1.set_xlabel('Latency (ms)')
ax1.set_ylabel('Frequency')
ax1.set_title('Latency Distribution by Complexity')
ax1.legend()

# 2. Latency over time
ax2 = axes[0, 1]
traffic_analysis_df['hour'] = pd.to_datetime(traffic_analysis_df['timestamp']).dt.hour
hourly_latency = traffic_analysis_df.groupby('hour')['latency_ms'].mean()
ax2.plot(hourly_latency.index, hourly_latency.values, marker='o')
ax2.set_xlabel('Hour of Day')
ax2.set_ylabel('Average Latency (ms)')
ax2.set_title('Average Latency by Hour')
ax2.grid(True, alpha=0.3)

# 3. Token usage vs latency
ax3 = axes[1, 0]
ax3.scatter(traffic_analysis_df['total_tokens'], traffic_analysis_df['latency_ms'],
            alpha=0.5, c=traffic_analysis_df['complexity'].map({'simple': 0, 'moderate': 1, 'complex': 2}))
ax3.set_xlabel('Total Tokens')
ax3.set_ylabel('Latency (ms)')
ax3.set_title('Token Usage vs Latency')

# 4. Latency percentiles by category
ax4 = axes[1, 1]
category_stats = traffic_analysis_df.groupby('category')['latency_ms'].agg(['mean', 'median', lambda x: x.quantile(0.95)])
category_stats.columns = ['Mean', 'Median', 'P95']
category_stats.plot(kind='bar', ax=ax4)
ax4.set_xlabel('Category')
ax4.set_ylabel('Latency (ms)')
ax4.set_title('Latency Statistics by Category')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('/tmp/latency_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Latency analysis charts generated")

### Step 4.3: Identify Slow and Anomalous Queries

We'll identify queries that exceed our latency thresholds and investigate their characteristics.

In [0]:
# =============================================================================
# ANOMALY DETECTION IN LATENCY
# =============================================================================

# Define latency thresholds based on complexity
LATENCY_THRESHOLDS = {
    "simple": 500,      # 500ms for simple queries
    "moderate": 1000,   # 1s for moderate queries
    "complex": 2000     # 2s for complex queries
}

# Identify slow queries
def is_slow_query(row):
    threshold = LATENCY_THRESHOLDS.get(row['complexity'], 1000)
    return row['latency_ms'] > threshold

traffic_analysis_df['is_slow'] = traffic_analysis_df.apply(is_slow_query, axis=1)

# Analyze slow queries
slow_queries = traffic_analysis_df[traffic_analysis_df['is_slow']]

print(f"🐢 Slow Query Analysis:")
print(f"   Total Slow Queries: {len(slow_queries)} ({len(slow_queries)/len(traffic_analysis_df)*100:.1f}%)")
print(f"\n   By Complexity:")
print(slow_queries.groupby('complexity').size())
print(f"\n   By Category:")
print(slow_queries.groupby('category').size())

# Identify extreme outliers (> 3 standard deviations)
mean_latency = traffic_analysis_df['latency_ms'].mean()
std_latency = traffic_analysis_df['latency_ms'].std()
outlier_threshold = mean_latency + 3 * std_latency

outliers = traffic_analysis_df[traffic_analysis_df['latency_ms'] > outlier_threshold]
print(f"\n⚠️ Extreme Outliers (>{outlier_threshold:.0f}ms): {len(outliers)}")

if len(outliers) > 0:
    print("\n   Sample outlier queries:")
    for _, row in outliers.head(3).iterrows():
        print(f"   - {row['request_id']}: {row['latency_ms']:.0f}ms ({row['complexity']})")

### Step 4.4: RAG Grounding Analysis

**Learning Objective 3:** Analyze inference tables to diagnose latency, token usage, and **RAG grounding issues**.

In RAG (Retrieval-Augmented Generation) systems, grounding issues occur when:
- The model generates answers not supported by the retrieved context
- Retrieved context is irrelevant to the question
- The model ignores relevant context and hallucinates

We'll analyze our inference data to identify potential grounding problems.

In [0]:
# =============================================================================
# RAG GROUNDING ANALYSIS
# =============================================================================
# Analyze the relationship between context quality and answer quality

# Simulate grounding analysis on our evaluation data
# In production, this would come from inference tables with context logged

def analyze_grounding_signals(question: str, context: str, answer: str) -> dict:
    """
    Analyze signals that indicate potential grounding issues.

    Returns dict with grounding quality indicators.
    """
    signals = {}

    # Signal 1: Context length - very short context may lack information
    context_words = len(context.split())
    signals['context_word_count'] = context_words
    signals['context_too_short'] = context_words < 20

    # Signal 2: Answer length vs context length ratio
    answer_words = len(answer.split())
    signals['answer_word_count'] = answer_words
    signals['answer_context_ratio'] = answer_words / max(context_words, 1)
    signals['answer_longer_than_context'] = answer_words > context_words  # Red flag

    # Signal 3: Key term overlap (simple heuristic)
    question_terms = set(question.lower().split())
    context_terms = set(context.lower().split())
    answer_terms = set(answer.lower().split())

    # Remove common stop words
    stop_words = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'what', 'how', 'why', 'when', 'where', 'who'}
    question_terms -= stop_words
    context_terms -= stop_words
    answer_terms -= stop_words

    # Check if answer terms appear in context
    answer_in_context = len(answer_terms & context_terms) / max(len(answer_terms), 1)
    signals['answer_grounded_ratio'] = answer_in_context
    signals['potential_hallucination'] = answer_in_context < 0.3  # Less than 30% overlap

    # Signal 4: Question-context relevance
    question_in_context = len(question_terms & context_terms) / max(len(question_terms), 1)
    signals['context_relevance'] = question_in_context
    signals['irrelevant_context'] = question_in_context < 0.2

    return signals

# Analyze grounding for our evaluation samples
grounding_analysis = []

for _, row in eval_results_df.iterrows():
    signals = analyze_grounding_signals(
        row['question'],
        row['context'],
        row['output']
    )
    signals['question'] = row['question'][:50] + "..."
    signals['groundedness_score'] = row.get('groundedness_score', 3)
    grounding_analysis.append(signals)

grounding_df = pd.DataFrame(grounding_analysis)

# Summary statistics
print("📊 RAG Grounding Analysis Summary:")
print(f"   Total samples analyzed: {len(grounding_df)}")
print(f"\n   Potential Issues Detected:")
print(f"   - Context too short (<20 words): {grounding_df['context_too_short'].sum()}")
print(f"   - Answer longer than context: {grounding_df['answer_longer_than_context'].sum()}")
print(f"   - Potential hallucinations: {grounding_df['potential_hallucination'].sum()}")
print(f"   - Irrelevant context retrieved: {grounding_df['irrelevant_context'].sum()}")

# Correlation with groundedness scores
if 'groundedness_score' in grounding_df.columns:
    print(f"\n   Average Groundedness by Issue Type:")
    print(f"   - With potential hallucination: {grounding_df[grounding_df['potential_hallucination']]['groundedness_score'].mean():.2f}")
    print(f"   - Without potential hallucination: {grounding_df[~grounding_df['potential_hallucination']]['groundedness_score'].mean():.2f}")

### Step 4.5: Distinguishing Retrieval Failures from Model/Prompt Failures

**Learning Objective 4:** Distinguish retrieval failures from model or prompt failures using inference logs.

When a RAG system produces poor answers, the root cause can be:

| Failure Type | Symptoms | Root Cause | Fix |
|--------------|----------|------------|-----|
| **Retrieval Failure** | Low context relevance, missing key info | Vector search issues, poor embeddings | Improve retrieval, re-index |
| **Model Failure** | Ignores good context, hallucinates | Model limitations, context too long | Try different model, truncate context |
| **Prompt Failure** | Misunderstands task, wrong format | Unclear instructions | Refine system prompt |

We'll implement a diagnostic framework to classify failures.

In [0]:
# =============================================================================
# FAILURE CLASSIFICATION FRAMEWORK
# =============================================================================
# Classify failures as retrieval, model, or prompt issues

def classify_failure(
    question: str,
    context: str,
    answer: str,
    ground_truth: str,
    groundedness_score: int,
    correctness_score: int
) -> dict:
    """
    Classify the type of failure based on evaluation signals.

    Returns classification with confidence and recommended action.
    """
    # Calculate diagnostic signals
    signals = analyze_grounding_signals(question, context, answer)

    classification = {
        "failure_type": None,
        "confidence": 0.0,
        "signals": [],
        "recommended_action": None
    }

    # Decision tree for classification

    # Case 1: Retrieval Failure
    # - Context is irrelevant to question
    # - Groundedness is low because context doesn't help
    if signals['irrelevant_context']:
        classification["failure_type"] = "RETRIEVAL_FAILURE"
        classification["confidence"] = 0.8
        classification["signals"].append("Context has low relevance to question")
        classification["recommended_action"] = "Improve retrieval: check embeddings, expand search, verify index"

    # Case 2: Model Failure (Hallucination)
    # - Context is relevant but answer doesn't use it
    # - Low groundedness despite good context
    elif signals['potential_hallucination'] and not signals['irrelevant_context']:
        classification["failure_type"] = "MODEL_FAILURE"
        classification["confidence"] = 0.7
        classification["signals"].append("Answer not grounded despite relevant context")
        classification["recommended_action"] = "Model issue: try larger model, reduce context length, add grounding instructions"

    # Case 3: Prompt Failure
    # - Answer is grounded but incorrect
    # - Model understood context but misunderstood task
    elif groundedness_score >= 4 and correctness_score <= 2:
        classification["failure_type"] = "PROMPT_FAILURE"
        classification["confidence"] = 0.6
        classification["signals"].append("Answer is grounded but incorrect - task misunderstanding")
        classification["recommended_action"] = "Refine prompt: clarify instructions, add examples, specify format"

    # Case 4: No clear failure
    else:
        classification["failure_type"] = "NO_CLEAR_FAILURE"
        classification["confidence"] = 0.5
        classification["signals"].append("No obvious failure pattern detected")
        classification["recommended_action"] = "Review manually or collect more data"

    return classification

# Classify failures in our evaluation data
failure_classifications = []

for _, row in eval_results_df.iterrows():
    classification = classify_failure(
        question=row['question'],
        context=row['context'],
        answer=row['output'],
        ground_truth=row['ground_truth'],
        groundedness_score=row.get('groundedness_score', 3),
        correctness_score=row.get('correctness_score', 3)
    )
    classification['question'] = row['question'][:40] + "..."
    failure_classifications.append(classification)

failure_df = pd.DataFrame(failure_classifications)

# Summary
print("🔍 Failure Classification Summary:")
print(f"\n   Failure Type Distribution:")
for failure_type in failure_df['failure_type'].unique():
    count = len(failure_df[failure_df['failure_type'] == failure_type])
    pct = count / len(failure_df) * 100
    print(f"   - {failure_type}: {count} ({pct:.1f}%)")

# Show examples of each failure type
print("\n📋 Example Failures by Type:")
for failure_type in ['RETRIEVAL_FAILURE', 'MODEL_FAILURE', 'PROMPT_FAILURE']:
    examples = failure_df[failure_df['failure_type'] == failure_type].head(1)
    if len(examples) > 0:
        ex = examples.iloc[0]
        print(f"\n   {failure_type}:")
        print(f"   Question: {ex['question']}")
        print(f"   Signals: {', '.join(ex['signals'])}")
        print(f"   Action: {ex['recommended_action']}")

---

# Part 5: Monitoring Dashboards and Agent Traces

In this section, we will:
1. Create monitoring dashboards for key metrics
2. Implement agent tracing for multi-step workflows
3. Analyze trace data to identify bottlenecks
4. Set up baseline-aware monitoring

Effective monitoring requires both real-time dashboards and detailed traces for debugging.

---

### Step 5.1: Create Monitoring Dashboard Metrics

We'll define key metrics for our monitoring dashboard following Chapter 8 best practices:

- **Latency metrics**: P50, P95, P99 response times
- **Quality metrics**: Groundedness, correctness scores
- **Cost metrics**: Token usage, request volume
- **Error metrics**: Error rates, timeout rates

In [0]:
# =============================================================================
# MONITORING DASHBOARD METRICS
# =============================================================================
# Define and calculate key monitoring metrics

def calculate_monitoring_metrics(df):
    """
    Calculate comprehensive monitoring metrics from traffic data.
    These metrics form the basis of our monitoring dashboard.
    """
    metrics = {
        # Latency Metrics
        "latency_p50": df['latency_ms'].median(),
        "latency_p95": df['latency_ms'].quantile(0.95),
        "latency_p99": df['latency_ms'].quantile(0.99),
        "latency_mean": df['latency_ms'].mean(),
        "latency_std": df['latency_ms'].std(),

        # Volume Metrics
        "total_requests": len(df),
        "requests_per_hour": len(df) / 168,  # 7 days * 24 hours
        "success_rate": (df['status'] == 'success').mean(),
        "error_rate": (df['status'] == 'error').mean(),

        # Token Metrics
        "avg_input_tokens": df['input_tokens'].mean(),
        "avg_output_tokens": df['output_tokens'].mean(),
        "avg_total_tokens": df['total_tokens'].mean(),
        "total_tokens_consumed": df['total_tokens'].sum(),

        # Quality Metrics (from our synthetic data)
        "avg_quality_score": df['quality_score'].mean(),
        "quality_below_threshold": (df['quality_score'] < 0.7).mean(),

        # Complexity Distribution
        "pct_simple": (df['complexity'] == 'simple').mean(),
        "pct_moderate": (df['complexity'] == 'moderate').mean(),
        "pct_complex": (df['complexity'] == 'complex').mean(),
    }

    return metrics

# Calculate current metrics
current_metrics = calculate_monitoring_metrics(traffic_analysis_df)

print("📊 Current Monitoring Metrics:")
print("\n   Latency Metrics:")
print(f"      P50: {current_metrics['latency_p50']:.2f} ms")
print(f"      P95: {current_metrics['latency_p95']:.2f} ms")
print(f"      P99: {current_metrics['latency_p99']:.2f} ms")

print("\n   Volume Metrics:")
print(f"      Total Requests: {current_metrics['total_requests']}")
print(f"      Success Rate: {current_metrics['success_rate']*100:.1f}%")

print("\n   Token Metrics:")
print(f"      Avg Total Tokens: {current_metrics['avg_total_tokens']:.1f}")
print(f"      Total Consumed: {current_metrics['total_tokens_consumed']:,}")

print("\n   Quality Metrics:")
print(f"      Avg Quality Score: {current_metrics['avg_quality_score']:.3f}")
print(f"      Below Threshold: {current_metrics['quality_below_threshold']*100:.1f}%")

### Step 5.2: Implement Agent Tracing

For multi-step agent workflows, we need detailed tracing to understand:
- Which steps take the longest
- Where errors occur
- How context flows between steps

We'll use MLflow's tracing capabilities to instrument our Q&A workflow.

In [0]:
import uuid

# =============================================================================
# AGENT TRACING IMPLEMENTATION
# =============================================================================
# Implement manual tracing for multi-step Q&A workflow
# We'll track timing and metrics for each step without relying on decorators

class TracedQAAgent:
    """
    A traced Q&A agent that logs detailed execution information.
    This simulates a multi-step agent workflow with:
    1. Query preprocessing
    2. Context retrieval
    3. LLM inference
    4. Response post-processing
    """

    def __init__(self, endpoint_name, client):
        self.endpoint_name = endpoint_name
        self.client = client
        self.trace_log = []

    def preprocess_query(self, question):
        """Step 1: Preprocess the user query"""
        start = time.time()

        # Simulate preprocessing (normalization, spell check, etc.)
        processed = question.strip().lower()
        processed = ' '.join(processed.split())  # Normalize whitespace

        latency = (time.time() - start) * 1000
        return {
            "original": question,
            "processed": processed,
            "latency_ms": latency
        }

    def retrieve_context(self, query, context):
        """Step 2: Retrieve relevant context"""
        start = time.time()

        # In production, this would query a vector database
        # For this lab, we use the provided context
        relevant_context = context

        latency = (time.time() - start) * 1000
        return {
            "context": relevant_context,
            "context_tokens": len(relevant_context.split()),
            "latency_ms": latency
        }

    def generate_response(self, question, context):
        """Step 3: Generate response using LLM"""
        start = time.time()

        system_prompt = """You are a helpful assistant. Answer based on the context provided."""
        user_prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"

        response = self.client.chat.completions.create(
            model=self.endpoint_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=300,
            temperature=0.1
        )

        latency = (time.time() - start) * 1000
        return {
            "response": response.choices[0].message.content,
            "tokens": response.usage.total_tokens,
            "latency_ms": latency
        }

    def postprocess_response(self, response):
        """Step 4: Post-process the response"""
        start = time.time()

        # Simulate post-processing (formatting, safety checks, etc.)
        processed = response.strip()

        latency = (time.time() - start) * 1000
        return {
            "final_response": processed,
            "latency_ms": latency
        }

    def run(self, question, context):
        """Execute the full Q&A pipeline with manual tracing"""
        trace_id = str(uuid.uuid4())[:8]
        trace_record = {"trace_id": trace_id, "steps": []}

        # Step 1: Preprocess
        preprocessed = self.preprocess_query(question)
        trace_record["steps"].append({"step": "preprocess", "latency_ms": preprocessed["latency_ms"]})

        # Step 2: Retrieve context
        retrieved = self.retrieve_context(preprocessed["processed"], context)
        trace_record["steps"].append({"step": "retrieve", "latency_ms": retrieved["latency_ms"]})

        # Step 3: Generate response
        generated = self.generate_response(question, retrieved["context"])
        trace_record["steps"].append({"step": "generate", "latency_ms": generated["latency_ms"]})

        # Step 4: Post-process
        final = self.postprocess_response(generated["response"])
        trace_record["steps"].append({"step": "postprocess", "latency_ms": final["latency_ms"]})

        # Calculate total latency
        total_latency = sum(step["latency_ms"] for step in trace_record["steps"])
        trace_record["total_latency_ms"] = total_latency

        # Store trace
        self.trace_log.append(trace_record)

        return {
            "trace_id": trace_id,
            "answer": final["final_response"],
            "total_latency_ms": total_latency,
            "step_latencies": {
                "preprocess": preprocessed["latency_ms"],
                "retrieve": retrieved["latency_ms"],
                "generate": generated["latency_ms"],
                "postprocess": final["latency_ms"]
            },
            "tokens": generated["tokens"]
        }

# Initialize traced agent
traced_agent = TracedQAAgent(ENDPOINT_NAME, client)
print("✅ Traced Q&A Agent initialized")

#### Running Traced Queries

**What we're doing:** Executing queries through our `TracedQAAgent` to collect detailed timing data for each step of the pipeline.

**How the tracing works:**

1. **Trace ID generation**: Each query gets a unique ID for correlation:
   ```python
   trace_id = str(uuid.uuid4())[:8]  # e.g., "a1b2c3d4"
   ```

2. **Step-by-step timing**: Each pipeline step is timed:
   ```python
   start = time.time()
   result = self.preprocess_query(question)
   latency_ms = (time.time() - start) * 1000
   trace_record["steps"].append({"step": "preprocess", "latency_ms": latency_ms})
   ```

3. **Pipeline steps traced**:
   | Step | What it does | Typical time |
   |------|-------------|--------------|
   | preprocess | Clean/normalize question | 1-5ms |
   | retrieve | Find relevant context | 5-20ms |
   | generate | LLM API call | 200-2000ms |
   | postprocess | Format response | 1-5ms |

4. **Trace log storage**: All traces stored in `traced_agent.trace_log` for analysis

**Why trace?** The LLM call typically dominates latency (>95%), but tracing helps identify when other steps become bottlenecks (e.g., slow retrieval from a vector database).

In [0]:
# =============================================================================
# RUN TRACED QUERIES
# =============================================================================
# Execute queries with manual tracing

print("🔄 Running traced queries...")

traced_results = []

# Run a few traced queries
for sample in eval_data[:5]:
    try:
        result = traced_agent.run(sample["question"], sample["context"])

        traced_results.append({
            "id": sample["id"],
            "complexity": sample["complexity"],
            "trace_id": result["trace_id"],
            **result["step_latencies"],
            "total_latency": result["total_latency_ms"],
            "tokens": result["tokens"]
        })

        print(f"   ✓ {sample['id']} [trace:{result['trace_id']}]: {result['total_latency_ms']:.0f}ms total")

    except Exception as e:
        print(f"   ✗ {sample['id']}: {str(e)}")

    time.sleep(1)

# Analyze step latencies
traced_df = pd.DataFrame(traced_results)

print(f"\n📊 Step Latency Analysis:")
print(f"   Preprocess:  {traced_df['preprocess'].mean():.2f}ms avg")
print(f"   Retrieve:    {traced_df['retrieve'].mean():.2f}ms avg")
print(f"   Generate:    {traced_df['generate'].mean():.2f}ms avg")
print(f"   Postprocess: {traced_df['postprocess'].mean():.2f}ms avg")
print(f"\n   LLM generation accounts for {traced_df['generate'].mean()/traced_df['total_latency'].mean()*100:.1f}% of total latency")

# Display trace log from agent
print(f"\n📋 Trace Log ({len(traced_agent.trace_log)} traces recorded):")
for trace in traced_agent.trace_log[:3]:
    print(f"   Trace {trace['trace_id']}: {trace['total_latency_ms']:.0f}ms")
    for step in trace['steps']:
        print(f"      └─ {step['step']}: {step['latency_ms']:.2f}ms")

---

# Part 6: Anomaly Detection and Alerting

In this section, we will:
1. Implement statistical anomaly detection
2. Configure alert thresholds based on baselines
3. Create alerting rules for production monitoring
4. Apply calibrated alerting strategies from Chapter 8

Effective alerting requires careful calibration to avoid alert fatigue while catching real issues.

---

### Step 6.1: Statistical Anomaly Detection

We'll implement anomaly detection using statistical methods:
- **Z-score detection**: Identify values beyond N standard deviations
- **IQR method**: Detect outliers using interquartile range
- **Rolling window analysis**: Detect trend changes over time

In [0]:
# =============================================================================
# ANOMALY DETECTION IMPLEMENTATION
# =============================================================================

class AnomalyDetector:
    """
    Statistical anomaly detection for LLM monitoring metrics.
    Implements multiple detection methods for robust alerting.
    """

    def __init__(self, baseline_df):
        """Initialize with baseline data for threshold calculation."""
        self.baseline = baseline_df
        self.thresholds = self._calculate_thresholds()

    def _calculate_thresholds(self):
        """Calculate detection thresholds from baseline data."""
        latency = self.baseline['latency_ms']
        tokens = self.baseline['total_tokens']

        return {
            'latency': {
                'mean': latency.mean(),
                'std': latency.std(),
                'p95': latency.quantile(0.95),
                'p99': latency.quantile(0.99),
                'iqr_low': latency.quantile(0.25) - 1.5 * (latency.quantile(0.75) - latency.quantile(0.25)),
                'iqr_high': latency.quantile(0.75) + 1.5 * (latency.quantile(0.75) - latency.quantile(0.25))
            },
            'tokens': {
                'mean': tokens.mean(),
                'std': tokens.std(),
                'p95': tokens.quantile(0.95),
                'p99': tokens.quantile(0.99)
            },
            'error_rate': {
                'baseline': (self.baseline['status'] == 'error').mean(),
                'threshold': max(0.05, (self.baseline['status'] == 'error').mean() * 2)
            }
        }

    def detect_latency_anomaly(self, value, method='zscore', threshold=3):
        """Detect if a latency value is anomalous."""
        if method == 'zscore':
            z = (value - self.thresholds['latency']['mean']) / self.thresholds['latency']['std']
            return abs(z) > threshold, z
        elif method == 'percentile':
            return value > self.thresholds['latency']['p99'], value / self.thresholds['latency']['p99']
        elif method == 'iqr':
            return value > self.thresholds['latency']['iqr_high'], value / self.thresholds['latency']['iqr_high']

    def detect_token_anomaly(self, value, threshold=2):
        """Detect if token usage is anomalous."""
        z = (value - self.thresholds['tokens']['mean']) / self.thresholds['tokens']['std']
        return abs(z) > threshold, z

    def detect_error_rate_anomaly(self, current_error_rate):
        """Detect if error rate exceeds threshold."""
        return current_error_rate > self.thresholds['error_rate']['threshold'], current_error_rate

    def analyze_batch(self, df):
        """Analyze a batch of data for anomalies."""
        anomalies = {
            'latency_anomalies': [],
            'token_anomalies': [],
            'error_rate_anomaly': False
        }

        for idx, row in df.iterrows():
            is_latency_anomaly, score = self.detect_latency_anomaly(row['latency_ms'])
            if is_latency_anomaly:
                anomalies['latency_anomalies'].append({
                    'request_id': row['request_id'],
                    'latency_ms': row['latency_ms'],
                    'z_score': score
                })

            is_token_anomaly, score = self.detect_token_anomaly(row['total_tokens'])
            if is_token_anomaly:
                anomalies['token_anomalies'].append({
                    'request_id': row['request_id'],
                    'total_tokens': row['total_tokens'],
                    'z_score': score
                })

        # Check overall error rate
        current_error_rate = (df['status'] == 'error').mean()
        anomalies['error_rate_anomaly'], _ = self.detect_error_rate_anomaly(current_error_rate)
        anomalies['current_error_rate'] = current_error_rate

        return anomalies

# Initialize detector with baseline data
detector = AnomalyDetector(traffic_analysis_df)

print("✅ Anomaly Detector initialized with baseline thresholds:")
print(f"\n   Latency Thresholds:")
print(f"      Mean: {detector.thresholds['latency']['mean']:.2f} ms")
print(f"      Std: {detector.thresholds['latency']['std']:.2f} ms")
print(f"      P95: {detector.thresholds['latency']['p95']:.2f} ms")
print(f"      P99: {detector.thresholds['latency']['p99']:.2f} ms")
print(f"\n   Token Thresholds:")
print(f"      Mean: {detector.thresholds['tokens']['mean']:.1f}")
print(f"      P95: {detector.thresholds['tokens']['p95']:.1f}")
print(f"\n   Error Rate Threshold: {detector.thresholds['error_rate']['threshold']*100:.1f}%")

#### Running Anomaly Detection

**What we're doing:** Scanning our traffic data to find requests that deviate significantly from normal patterns.

**How the `analyze_batch()` method works:**

1. **Iterate through each request**:
   ```python
   for idx, row in df.iterrows():
       is_anomaly, z_score = self.detect_latency_anomaly(row['latency_ms'])
   ```

2. **Z-score calculation** for each metric:
   ```python
   z_score = (value - mean) / std_dev
   # If z_score > 3, the value is 3+ standard deviations from mean
   ```

3. **Anomaly types detected**:

   | Type | Detection Method | Threshold |
   |------|-----------------|-----------|
   | Latency | Z-score > 3 | ~99.7% of normal data is below this |
   | Tokens | Z-score > 2 | ~95% of normal data is below this |
   | Error Rate | Rate > 5% | Fixed threshold based on SLA |

4. **Output structure**:
   ```python
   {
       'latency_anomalies': [{'request_id': 'x', 'latency_ms': 5000, 'z_score': 4.2}],
       'token_anomalies': [...],
       'error_rate_anomaly': True/False,
       'current_error_rate': 0.03
   }
   ```

**Why Z-scores?** They're distribution-agnostic and automatically adapt to your baseline. A Z-score of 3 means "this is extremely unusual" regardless of whether your mean latency is 100ms or 1000ms.

In [0]:
# =============================================================================
# RUN ANOMALY DETECTION
# =============================================================================

# Analyze our traffic data for anomalies
anomalies = detector.analyze_batch(traffic_analysis_df)

print("🔍 Anomaly Detection Results:")
print(f"\n   Latency Anomalies: {len(anomalies['latency_anomalies'])}")
if anomalies['latency_anomalies']:
    print("   Top 5 latency anomalies:")
    for a in sorted(anomalies['latency_anomalies'], key=lambda x: x['z_score'], reverse=True)[:5]:
        print(f"      {a['request_id']}: {a['latency_ms']:.0f}ms (z={a['z_score']:.2f})")

print(f"\n   Token Anomalies: {len(anomalies['token_anomalies'])}")
if anomalies['token_anomalies']:
    print("   Top 5 token anomalies:")
    for a in sorted(anomalies['token_anomalies'], key=lambda x: x['z_score'], reverse=True)[:5]:
        print(f"      {a['request_id']}: {a['total_tokens']} tokens (z={a['z_score']:.2f})")

print(f"\n   Error Rate Anomaly: {'⚠️ YES' if anomalies['error_rate_anomaly'] else '✅ NO'}")
print(f"   Current Error Rate: {anomalies['current_error_rate']*100:.2f}%")

### Step 6.2: Configure Alert Rules

Following Chapter 8 best practices for calibrated alerting:
- **Severity levels**: Critical, Warning, Info
- **Cooldown periods**: Prevent alert storms
- **Aggregation windows**: Reduce noise from transient spikes

In [0]:
# =============================================================================
# ALERT CONFIGURATION
# =============================================================================

class AlertManager:
    """
    Alert management system with calibrated thresholds and cooldowns.
    Implements Chapter 8 best practices for production alerting.
    """

    def __init__(self, detector):
        self.detector = detector
        self.alert_history = []
        self.cooldown_periods = {
            'critical': 300,   # 5 minutes
            'warning': 900,    # 15 minutes
            'info': 3600       # 1 hour
        }

        # Define alert rules
        self.rules = {
            'latency_critical': {
                'condition': lambda m: m['latency_p99'] > detector.thresholds['latency']['p99'] * 2,
                'severity': 'critical',
                'message': 'P99 latency exceeds 2x baseline'
            },
            'latency_warning': {
                'condition': lambda m: m['latency_p95'] > detector.thresholds['latency']['p95'] * 1.5,
                'severity': 'warning',
                'message': 'P95 latency exceeds 1.5x baseline'
            },
            'error_rate_critical': {
                'condition': lambda m: m['error_rate'] > 0.10,
                'severity': 'critical',
                'message': 'Error rate exceeds 10%'
            },
            'error_rate_warning': {
                'condition': lambda m: m['error_rate'] > 0.05,
                'severity': 'warning',
                'message': 'Error rate exceeds 5%'
            },
            'token_spike': {
                'condition': lambda m: m['avg_total_tokens'] > detector.thresholds['tokens']['p95'],
                'severity': 'warning',
                'message': 'Average token usage exceeds P95 baseline'
            },
            'quality_degradation': {
                'condition': lambda m: m.get('avg_quality_score', 1) < 0.7,
                'severity': 'warning',
                'message': 'Average quality score below threshold'
            }
        }

    def evaluate_rules(self, metrics):
        """Evaluate all alert rules against current metrics."""
        triggered_alerts = []

        for rule_name, rule in self.rules.items():
            try:
                if rule['condition'](metrics):
                    triggered_alerts.append({
                        'rule': rule_name,
                        'severity': rule['severity'],
                        'message': rule['message'],
                        'timestamp': datetime.now()
                    })
            except Exception as e:
                print(f"Error evaluating rule {rule_name}: {e}")

        return triggered_alerts

    def should_alert(self, alert):
        """Check if alert should fire based on cooldown."""
        cooldown = self.cooldown_periods[alert['severity']]

        # Check recent alerts of same type
        for hist in self.alert_history:
            if hist['rule'] == alert['rule']:
                time_diff = (alert['timestamp'] - hist['timestamp']).total_seconds()
                if time_diff < cooldown:
                    return False

        return True

    def process_alerts(self, metrics):
        """Process metrics and generate alerts."""
        triggered = self.evaluate_rules(metrics)
        fired_alerts = []

        for alert in triggered:
            if self.should_alert(alert):
                fired_alerts.append(alert)
                self.alert_history.append(alert)

        return fired_alerts

# Initialize alert manager
alert_manager = AlertManager(detector)

# Evaluate current metrics
alerts = alert_manager.process_alerts(current_metrics)

print("🚨 Alert Evaluation Results:")
if alerts:
    for alert in alerts:
        severity_icon = {'critical': '🔴', 'warning': '🟡', 'info': '🔵'}
        print(f"   {severity_icon[alert['severity']]} [{alert['severity'].upper()}] {alert['message']}")
else:
    print("   ✅ No alerts triggered - all metrics within normal range")

print(f"\n📋 Configured Alert Rules:")
for rule_name, rule in alert_manager.rules.items():
    print(f"   - {rule_name} ({rule['severity']}): {rule['message']}")

---

# Part 7: Cost Optimization Strategies

In this section, we will:
1. Analyze token usage patterns for cost drivers
2. Implement prompt optimization techniques
3. Apply context control strategies
4. Validate improvements with evaluation runs

Cost optimization is critical for sustainable LLM operations at scale.

---

### Step 7.1: Analyze Cost Drivers

We'll identify the main contributors to token usage and cost:
- Long context documents
- Verbose prompts
- Unnecessarily detailed responses

In [0]:
# =============================================================================
# COST ANALYSIS
# =============================================================================

# Analyze token usage patterns
print("💰 Cost Analysis:")

# Token usage by complexity
token_by_complexity = traffic_analysis_df.groupby('complexity').agg({
    'input_tokens': 'mean',
    'output_tokens': 'mean',
    'total_tokens': ['mean', 'sum']
}).round(2)

print("\n   Token Usage by Complexity:")
print(token_by_complexity)

# Token usage by category
token_by_category = traffic_analysis_df.groupby('category').agg({
    'total_tokens': ['mean', 'sum', 'count']
}).round(2)

print("\n   Token Usage by Category:")
print(token_by_category)

# Identify high-cost queries
high_cost_threshold = traffic_analysis_df['total_tokens'].quantile(0.9)
high_cost_queries = traffic_analysis_df[traffic_analysis_df['total_tokens'] > high_cost_threshold]

print(f"\n   High-Cost Queries (>{high_cost_threshold:.0f} tokens): {len(high_cost_queries)}")
print(f"   These represent {len(high_cost_queries)/len(traffic_analysis_df)*100:.1f}% of requests")
print(f"   But consume {high_cost_queries['total_tokens'].sum()/traffic_analysis_df['total_tokens'].sum()*100:.1f}% of tokens")

# Estimate costs (using approximate pricing)
COST_PER_1K_INPUT_TOKENS = 0.0015  # Example pricing
COST_PER_1K_OUTPUT_TOKENS = 0.002

total_input_tokens = traffic_analysis_df['input_tokens'].sum()
total_output_tokens = traffic_analysis_df['output_tokens'].sum()

estimated_cost = (total_input_tokens / 1000 * COST_PER_1K_INPUT_TOKENS +
                  total_output_tokens / 1000 * COST_PER_1K_OUTPUT_TOKENS)

print(f"\n   Estimated Cost for {len(traffic_analysis_df)} requests: ${estimated_cost:.2f}")
print(f"   Average Cost per Request: ${estimated_cost/len(traffic_analysis_df)*1000:.4f}")

### Step 7.2: Implement Prompt Optimization

We'll implement several prompt optimization techniques:
1. **Context truncation**: Limit context to relevant portions
2. **Prompt compression**: Remove redundant instructions
3. **Response length control**: Set appropriate max_tokens

In [0]:
# =============================================================================
# PROMPT OPTIMIZATION TECHNIQUES
# =============================================================================

class PromptOptimizer:
    """
    Implements prompt optimization strategies for cost reduction.
    """

    def __init__(self, max_context_tokens=200, max_response_tokens=150):
        self.max_context_tokens = max_context_tokens
        self.max_response_tokens = max_response_tokens

    def truncate_context(self, context, max_tokens=None):
        """Truncate context to maximum token limit."""
        max_tokens = max_tokens or self.max_context_tokens
        words = context.split()

        # Approximate: 1 token ≈ 0.75 words
        max_words = int(max_tokens * 0.75)

        if len(words) <= max_words:
            return context, False

        truncated = ' '.join(words[:max_words]) + '...'
        return truncated, True

    def compress_prompt(self, system_prompt, user_prompt):
        """Compress prompts by removing redundancy."""
        # Remove excessive whitespace
        system_prompt = ' '.join(system_prompt.split())
        user_prompt = ' '.join(user_prompt.split())

        # Use shorter system prompt
        compressed_system = "Answer questions based on the context. Be concise."

        return compressed_system, user_prompt

    def optimize_query(self, question, context):
        """Apply all optimization techniques."""
        # Truncate context
        truncated_context, was_truncated = self.truncate_context(context)

        # Create optimized prompts
        system_prompt = "Answer based on context. Be concise and accurate."
        user_prompt = f"Context: {truncated_context}\n\nQ: {question}\n\nA:"

        return {
            'system_prompt': system_prompt,
            'user_prompt': user_prompt,
            'context_truncated': was_truncated,
            'max_tokens': self.max_response_tokens
        }

# Initialize optimizer
optimizer = PromptOptimizer(max_context_tokens=150, max_response_tokens=100)

# Test optimization on a sample
sample = eval_data[0]
original_context_tokens = len(sample['context'].split())
optimized = optimizer.optimize_query(sample['question'], sample['context'])

print("🔧 Prompt Optimization Example:")
print(f"\n   Original context tokens: ~{original_context_tokens}")
print(f"   Optimized context tokens: ~{len(optimized['user_prompt'].split())}")
print(f"   Context truncated: {optimized['context_truncated']}")
print(f"   Max response tokens: {optimized['max_tokens']}")
print(f"\n   Optimized system prompt: '{optimized['system_prompt']}'")

#### Comparing Optimized vs Baseline Performance

**What we're doing:** Running the same queries with optimized prompts and comparing token usage, latency, and answer quality against our baseline.

**How the comparison works:**

1. **Run both versions** on the same questions:
   ```python
   baseline_result = query_qa_endpoint(question, context)  # Full prompts
   optimized_result = query_optimized(question, context, optimizer)  # Compressed
   ```

2. **Metrics compared**:
   | Metric | Baseline | Optimized | Goal |
   |--------|----------|-----------|------|
   | Input tokens | ~300 | ~150 | 50% reduction |
   | Output tokens | ~200 | ~100 | 50% reduction |
   | Latency | ~500ms | ~350ms | Lower is better |
   | Quality | 0.85 | 0.80+ | Minimal degradation |

3. **Cost calculation**:
   ```python
   # Typical pricing: $0.002 per 1K tokens
   baseline_cost = baseline_tokens * 0.002 / 1000
   optimized_cost = optimized_tokens * 0.002 / 1000
   savings = (baseline_cost - optimized_cost) / baseline_cost * 100
   ```

**The trade-off:** Aggressive optimization saves money but may reduce answer quality. The goal is finding the sweet spot where you save 30-50% on tokens while maintaining >95% of baseline quality.

In [0]:
# =============================================================================
# COMPARE OPTIMIZED VS BASELINE PERFORMANCE
# =============================================================================

def query_optimized(question, context, optimizer):
    """Query endpoint with optimized prompts."""
    opt = optimizer.optimize_query(question, context)

    start_time = time.time()

    response = client.chat.completions.create(
        model=ENDPOINT_NAME,
        messages=[
            {"role": "system", "content": opt['system_prompt']},
            {"role": "user", "content": opt['user_prompt']}
        ],
        max_tokens=opt['max_tokens'],
        temperature=0.1
    )

    latency_ms = (time.time() - start_time) * 1000

    return {
        "answer": response.choices[0].message.content,
        "latency_ms": latency_ms,
        "input_tokens": response.usage.prompt_tokens,
        "output_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens
    }

print("🔄 Comparing baseline vs optimized performance...")

comparison_results = []

for sample in eval_data[:5]:
    try:
        # Baseline query
        baseline = query_qa_endpoint(sample["question"], sample["context"])

        # Optimized query
        optimized_result = query_optimized(sample["question"], sample["context"], optimizer)

        comparison_results.append({
            "id": sample["id"],
            "baseline_tokens": baseline["total_tokens"],
            "optimized_tokens": optimized_result["total_tokens"],
            "token_reduction": baseline["total_tokens"] - optimized_result["total_tokens"],
            "baseline_latency": baseline["latency_ms"],
            "optimized_latency": optimized_result["latency_ms"]
        })

        print(f"   ✓ {sample['id']}: {baseline['total_tokens']} → {optimized_result['total_tokens']} tokens")

    except Exception as e:
        print(f"   ✗ {sample['id']}: {str(e)}")

    time.sleep(1)

# Analyze results
comparison_df = pd.DataFrame(comparison_results)

print(f"\n📊 Optimization Results:")
print(f"   Average Token Reduction: {comparison_df['token_reduction'].mean():.1f} tokens ({comparison_df['token_reduction'].mean()/comparison_df['baseline_tokens'].mean()*100:.1f}%)")
print(f"   Average Latency Change: {(comparison_df['optimized_latency'].mean() - comparison_df['baseline_latency'].mean()):.1f}ms")

# Estimate cost savings
baseline_cost = comparison_df['baseline_tokens'].sum() / 1000 * 0.002
optimized_cost = comparison_df['optimized_tokens'].sum() / 1000 * 0.002
savings = baseline_cost - optimized_cost

print(f"\n   Estimated Cost Savings: ${savings:.4f} for {len(comparison_df)} queries")
print(f"   Projected Monthly Savings (10K queries/day): ${savings/len(comparison_df)*10000*30:.2f}")

---

# Part 8: Validation and Periodic Review Cycles

In this section, we will:
1. Validate optimizations with repeatable evaluation runs
2. Establish baseline metrics for ongoing comparison
3. Implement periodic review workflows
4. Create evaluation versioning for reproducibility

Following Chapter 8 best practices, we'll ensure our improvements are measurable and sustainable.

---

### Step 8.1: Validate Optimizations

We'll run a comprehensive evaluation to validate that our optimizations maintain quality while reducing costs.

In [0]:
# =============================================================================
# VALIDATION EVALUATION RUN
# =============================================================================

print("🔄 Running validation evaluation...")

validation_results = []

for sample in eval_data:
    try:
        # Run optimized query
        result = query_optimized(sample["question"], sample["context"], optimizer)

        validation_results.append({
            "id": sample["id"],
            "question": sample["question"],
            "context": sample["context"],
            "ground_truth": sample["ground_truth"],
            "output": result["answer"],
            "category": sample["category"],
            "complexity": sample["complexity"],
            "latency_ms": result["latency_ms"],
            "input_tokens": result["input_tokens"],
            "output_tokens": result["output_tokens"],
            "total_tokens": result["total_tokens"]
        })

    except Exception as e:
        print(f"   ✗ {sample['id']}: {str(e)}")

    time.sleep(0.5)

validation_df = pd.DataFrame(validation_results)

print(f"\n✅ Validation complete: {len(validation_df)} samples evaluated")

#### Logging Validation Results to MLflow

**What we're doing:** Creating a new MLflow run for our optimized model to enable side-by-side comparison with the baseline.

**How the comparison works in MLflow:**

1. **Separate runs for each configuration**:
   - `baseline_evaluation`: Original prompts, no optimization
   - `optimized_validation`: Compressed prompts, token limits

2. **Parameters logged** (what we changed):
   ```python
   mlflow.log_param("optimization_type", "prompt_compression")
   mlflow.log_param("max_context_tokens", 150)  # Was unlimited
   mlflow.log_param("max_response_tokens", 100)  # Was unlimited
   ```

3. **Metrics logged using new GenAI API**:
   ```python
   # Run evaluation functions on each sample
   g_score = evaluate_groundedness(question=..., context=..., answer=...)
   c_score = evaluate_correctness(question=..., ground_truth=..., answer=...)

   # Log aggregate metrics
   mlflow.log_metric("groundedness_mean", avg_groundedness)
   mlflow.log_metric("answer_correctness_mean", avg_correctness)
   ```

4. **MLflow UI comparison**:
   - Navigate to Experiments → Select both runs → Compare
   - See metrics side-by-side in charts
   - Identify if quality dropped with optimization

**Decision framework:**
- If quality drops <5% and tokens drop >30% → Accept optimization
- If quality drops >10% → Reject, try less aggressive settings
- If quality unchanged → Great! Deploy optimized version

In [0]:
# =============================================================================
# LOG VALIDATION RUN TO MLFLOW
# =============================================================================

with mlflow.start_run(run_name="optimized_validation") as run:

    # Log parameters
    mlflow.log_param("endpoint_name", ENDPOINT_NAME)
    mlflow.log_param("optimization_type", "prompt_compression")
    mlflow.log_param("max_context_tokens", optimizer.max_context_tokens)
    mlflow.log_param("max_response_tokens", optimizer.max_response_tokens)
    mlflow.log_param("num_samples", len(validation_df))

    # Run evaluation using new GenAI API scorers
    print("🔄 Running MLflow evaluation on optimized results...")
    print("   (Using new MLflow 3.4+ GenAI API)")

    # Prepare data and run scorers
    val_groundedness_scores = []
    val_correctness_scores = []

    for i, row in validation_df.iterrows():
        try:
            # Use the evaluation functions with explicit parameters
            g_score = evaluate_groundedness(
                question=row["question"],
                context=row["context"],
                answer=row["output"]
            )
            c_score = evaluate_correctness(
                question=row["question"],
                ground_truth=row["ground_truth"],
                answer=row["output"]
            )

            val_groundedness_scores.append(g_score)
            val_correctness_scores.append(c_score)
        except Exception as e:
            print(f"   Warning: Error evaluating sample {i}: {e}")
            val_groundedness_scores.append(3)
            val_correctness_scores.append(3)
        time.sleep(0.5)  # Rate limiting

    # Calculate and log aggregate metrics
    avg_groundedness = sum(val_groundedness_scores) / len(val_groundedness_scores)
    avg_correctness = sum(val_correctness_scores) / len(val_correctness_scores)

    mlflow.log_metric("groundedness_mean", avg_groundedness)
    mlflow.log_metric("answer_correctness_mean", avg_correctness)

    # Log performance metrics
    mlflow.log_metric("avg_latency_ms", validation_df["latency_ms"].mean())
    mlflow.log_metric("p95_latency_ms", validation_df["latency_ms"].quantile(0.95))
    mlflow.log_metric("avg_total_tokens", validation_df["total_tokens"].mean())
    mlflow.log_metric("total_tokens_consumed", validation_df["total_tokens"].sum())

    # Add scores to dataframe
    validation_df["groundedness_score"] = val_groundedness_scores
    validation_df["correctness_score"] = val_correctness_scores

    # Log artifacts
    validation_df.to_csv("/tmp/validation_results.csv", index=False)
    mlflow.log_artifact("/tmp/validation_results.csv")

    print(f"\n✅ Validation logged to MLflow")
    print(f"   Run ID: {run.info.run_id}")

    # Compare with baseline
    print(f"\n📊 Validation Metrics:")
    print(f"   groundedness_mean: {avg_groundedness:.2f}")
    print(f"   answer_correctness_mean: {avg_correctness:.2f}")
    print(f"   avg_latency_ms: {validation_df['latency_ms'].mean():.2f}")
    print(f"   avg_total_tokens: {validation_df['total_tokens'].mean():.1f}")

### Step 8.1.5: Making the Deployment Decision

**Learning Objective 2:** Compare multiple model or prompt configurations using MLflow evaluation metrics and **justify a deployment choice**.

After running evaluations on both baseline and optimized configurations, we need a structured framework to make deployment decisions. This ensures decisions are data-driven, documented, and defensible.

---

#### Deployment Decision Framework

| Criterion | Threshold | Weight | Rationale |
|-----------|-----------|--------|-----------|
| **Quality (Correctness)** | Max 5% degradation | 40% | User experience is paramount |
| **Groundedness** | Max 10% degradation | 25% | RAG systems must stay grounded |
| **Latency (P95)** | Max 20% increase | 20% | Performance affects UX |
| **Cost (Tokens)** | Min 20% reduction | 15% | Must justify optimization effort |

---

#### Decision Matrix

| Scenario | Quality Δ | Cost Δ | Decision | Justification |
|----------|-----------|--------|----------|---------------|
| Win-Win | ≤0% | ≥-30% | ✅ **DEPLOY** | Quality maintained, significant savings |
| Acceptable Trade-off | -5% | ≥-40% | ✅ **DEPLOY** | Minor quality loss offset by major savings |
| Marginal | -5% to -10% | -20% to -30% | ⚠️ **REVIEW** | Needs stakeholder approval |
| Unacceptable | >-10% | Any | ❌ **REJECT** | Quality degradation too high |
| No Benefit | ≤0% | <-10% | ❌ **REJECT** | Insufficient cost savings |

In [0]:
# =============================================================================
# DEPLOYMENT DECISION FRAMEWORK
# =============================================================================
# Compare baseline vs optimized and make data-driven deployment decision

def make_deployment_decision(baseline_metrics: dict, optimized_metrics: dict) -> dict:
    """
    Evaluate whether to deploy the optimized configuration.

    Returns decision with justification and confidence score.
    """
    decision = {
        "recommendation": None,
        "confidence": 0.0,
        "justification": [],
        "metrics_comparison": {},
        "risk_assessment": None
    }

    # Calculate deltas (negative = degradation for quality, improvement for cost)
    quality_delta = (
        (optimized_metrics.get('correctness_mean', 3) - baseline_metrics.get('correctness_mean', 3))
        / max(baseline_metrics.get('correctness_mean', 3), 1) * 100
    )

    groundedness_delta = (
        (optimized_metrics.get('groundedness_mean', 3) - baseline_metrics.get('groundedness_mean', 3))
        / max(baseline_metrics.get('groundedness_mean', 3), 1) * 100
    )

    latency_delta = (
        (optimized_metrics.get('latency_p95', 500) - baseline_metrics.get('latency_p95', 500))
        / max(baseline_metrics.get('latency_p95', 500), 1) * 100
    )

    cost_delta = (
        (optimized_metrics.get('avg_tokens', 200) - baseline_metrics.get('avg_tokens', 200))
        / max(baseline_metrics.get('avg_tokens', 200), 1) * 100
    )

    decision["metrics_comparison"] = {
        "quality_change_pct": quality_delta,
        "groundedness_change_pct": groundedness_delta,
        "latency_change_pct": latency_delta,
        "cost_change_pct": cost_delta
    }

    # Apply decision rules
    quality_ok = quality_delta >= -5
    groundedness_ok = groundedness_delta >= -10
    latency_ok = latency_delta <= 20
    cost_benefit = cost_delta <= -20  # At least 20% reduction

    # Decision logic
    if quality_delta > -1 and cost_delta <= -30:
        decision["recommendation"] = "DEPLOY"
        decision["confidence"] = 0.95
        decision["justification"].append("Win-win: Quality maintained with significant cost savings (>30%)")
        decision["risk_assessment"] = "LOW"

    elif quality_ok and groundedness_ok and cost_delta <= -40:
        decision["recommendation"] = "DEPLOY"
        decision["confidence"] = 0.85
        decision["justification"].append("Acceptable trade-off: Minor quality impact offset by major savings (>40%)")
        decision["risk_assessment"] = "LOW-MEDIUM"

    elif quality_ok and groundedness_ok and cost_benefit:
        decision["recommendation"] = "DEPLOY"
        decision["confidence"] = 0.75
        decision["justification"].append("Good trade-off: Quality within tolerance, meaningful cost reduction")
        decision["risk_assessment"] = "MEDIUM"

    elif quality_delta < -10 or groundedness_delta < -15:
        decision["recommendation"] = "REJECT"
        decision["confidence"] = 0.90
        decision["justification"].append("Quality degradation exceeds acceptable threshold")
        decision["risk_assessment"] = "HIGH"

    elif not cost_benefit:
        decision["recommendation"] = "REJECT"
        decision["confidence"] = 0.80
        decision["justification"].append("Insufficient cost savings to justify optimization effort")
        decision["risk_assessment"] = "N/A"

    else:
        decision["recommendation"] = "REVIEW"
        decision["confidence"] = 0.60
        decision["justification"].append("Marginal case - requires stakeholder review")
        decision["risk_assessment"] = "MEDIUM"

    # Add specific metric observations
    if quality_delta < 0:
        decision["justification"].append(f"Quality decreased by {abs(quality_delta):.1f}%")
    else:
        decision["justification"].append(f"Quality improved by {quality_delta:.1f}%")

    if cost_delta < 0:
        decision["justification"].append(f"Token usage reduced by {abs(cost_delta):.1f}%")

    return decision

# Calculate metrics from our evaluation runs
baseline_metrics = {
    "correctness_mean": eval_results_df.get('correctness_score', pd.Series([3]*len(eval_results_df))).mean(),
    "groundedness_mean": eval_results_df.get('groundedness_score', pd.Series([3]*len(eval_results_df))).mean(),
    "latency_p95": eval_results_df['latency_ms'].quantile(0.95),
    "avg_tokens": eval_results_df['total_tokens'].mean()
}

optimized_metrics = {
    "correctness_mean": validation_df.get('correctness_score', pd.Series([3]*len(validation_df))).mean(),
    "groundedness_mean": validation_df.get('groundedness_score', pd.Series([3]*len(validation_df))).mean(),
    "latency_p95": validation_df['latency_ms'].quantile(0.95),
    "avg_tokens": validation_df['total_tokens'].mean()
}

# Make deployment decision
deployment_decision = make_deployment_decision(baseline_metrics, optimized_metrics)

# Display decision report
print("=" * 60)
print("🚀 DEPLOYMENT DECISION REPORT")
print("=" * 60)

recommendation_icon = {
    "DEPLOY": "✅",
    "REJECT": "❌",
    "REVIEW": "⚠️"
}.get(deployment_decision["recommendation"], "❓")

print(f"\n{recommendation_icon} RECOMMENDATION: {deployment_decision['recommendation']}")
print(f"   Confidence: {deployment_decision['confidence']*100:.0f}%")
print(f"   Risk Level: {deployment_decision['risk_assessment']}")

print(f"\n📊 METRICS COMPARISON:")
for metric, value in deployment_decision["metrics_comparison"].items():
    direction = "↑" if value > 0 else "↓" if value < 0 else "→"
    print(f"   {metric}: {value:+.1f}% {direction}")

print(f"\n📝 JUSTIFICATION:")
for i, reason in enumerate(deployment_decision["justification"], 1):
    print(f"   {i}. {reason}")

print("\n" + "=" * 60)

# Log decision to MLflow
with mlflow.start_run(run_name="deployment_decision"):
    mlflow.log_param("recommendation", deployment_decision["recommendation"])
    mlflow.log_param("confidence", deployment_decision["confidence"])
    mlflow.log_param("risk_assessment", deployment_decision["risk_assessment"])
    for metric, value in deployment_decision["metrics_comparison"].items():
        mlflow.log_metric(metric, value)
    print("✅ Deployment decision logged to MLflow")

### Step 8.2: Establish Review Cycle Workflow

We'll create a structured review workflow that can be run periodically to:
- Compare current performance against baselines
- Detect drift in quality or performance
- Generate actionable reports

In [0]:
# =============================================================================
# PERIODIC REVIEW WORKFLOW
# =============================================================================

class PeriodicReviewWorkflow:
    """
    Implements a structured review workflow for LLM monitoring.
    Following Chapter 8 best practices for periodic evaluation.
    """

    def __init__(self, baseline_metrics, alert_manager):
        self.baseline = baseline_metrics
        self.alert_manager = alert_manager
        self.review_history = []

    def run_review(self, current_df, review_name="periodic_review"):
        """Execute a complete review cycle."""
        review_timestamp = datetime.now()

        # Calculate current metrics
        current_metrics = calculate_monitoring_metrics(current_df)

        # Compare with baseline
        comparison = self._compare_metrics(current_metrics)

        # Check for alerts
        alerts = self.alert_manager.evaluate_rules(current_metrics)

        # Generate recommendations
        recommendations = self._generate_recommendations(comparison, alerts)

        # Create review report
        report = {
            "review_name": review_name,
            "timestamp": review_timestamp,
            "current_metrics": current_metrics,
            "baseline_comparison": comparison,
            "alerts": alerts,
            "recommendations": recommendations,
            "status": "HEALTHY" if not alerts else "NEEDS_ATTENTION"
        }

        self.review_history.append(report)

        return report

    def _compare_metrics(self, current):
        """Compare current metrics against baseline."""
        comparison = {}

        for key in ['latency_p50', 'latency_p95', 'avg_total_tokens', 'error_rate']:
            if key in self.baseline and key in current:
                baseline_val = self.baseline[key]
                current_val = current[key]

                if baseline_val > 0:
                    pct_change = (current_val - baseline_val) / baseline_val * 100
                else:
                    pct_change = 0

                comparison[key] = {
                    "baseline": baseline_val,
                    "current": current_val,
                    "change_pct": pct_change,
                    "status": "OK" if abs(pct_change) < 20 else "DEGRADED" if pct_change > 0 else "IMPROVED"
                }

        return comparison

    def _generate_recommendations(self, comparison, alerts):
        """Generate actionable recommendations."""
        recommendations = []

        # Check latency
        if comparison.get('latency_p95', {}).get('status') == 'DEGRADED':
            recommendations.append({
                "priority": "HIGH",
                "area": "Latency",
                "action": "Investigate latency increase. Consider scaling up or optimizing prompts."
            })

        # Check token usage
        if comparison.get('avg_total_tokens', {}).get('status') == 'DEGRADED':
            recommendations.append({
                "priority": "MEDIUM",
                "area": "Cost",
                "action": "Token usage increased. Review prompt templates and context lengths."
            })

        # Check error rate
        if comparison.get('error_rate', {}).get('status') == 'DEGRADED':
            recommendations.append({
                "priority": "HIGH",
                "area": "Reliability",
                "action": "Error rate increased. Check endpoint health and input validation."
            })

        # Add alert-based recommendations
        for alert in alerts:
            if alert['severity'] == 'critical':
                recommendations.append({
                    "priority": "CRITICAL",
                    "area": alert['rule'],
                    "action": f"Address immediately: {alert['message']}"
                })

        return recommendations

# Initialize review workflow
review_workflow = PeriodicReviewWorkflow(current_metrics, alert_manager)

# Run a review
review_report = review_workflow.run_review(traffic_analysis_df, "lab_final_review")

print("📋 Periodic Review Report")
print("=" * 50)
print(f"\n   Review: {review_report['review_name']}")
print(f"   Timestamp: {review_report['timestamp']}")
print(f"   Status: {review_report['status']}")

print(f"\n   Baseline Comparison:")
for metric, data in review_report['baseline_comparison'].items():
    print(f"      {metric}: {data['current']:.2f} ({data['change_pct']:+.1f}%) - {data['status']}")

if review_report['recommendations']:
    print(f"\n   Recommendations:")
    for rec in review_report['recommendations']:
        print(f"      [{rec['priority']}] {rec['area']}: {rec['action']}")
else:
    print(f"\n   ✅ No recommendations - system performing within expected parameters")

#### Generating the Review Report

**What we're doing:** Creating a human-readable report that summarizes the review findings for stakeholders who may not be technical.

**Report structure:**

1. **Executive Summary**: High-level health status
   - Overall health: HEALTHY / NEEDS ATTENTION / CRITICAL
   - Review period and sample size
   - One-line summary of key findings

2. **Key Metrics Table**: Current vs baseline comparison
   ```
   ✅ latency_p50: 245ms (baseline: 240ms, change: +2.1%)
   ⚠️ latency_p95: 890ms (baseline: 720ms, change: +23.6%)
   ✅ error_rate: 1.2% (baseline: 1.5%, change: -20.0%)
   ```

3. **Active Alerts**: Any triggered alert rules
   - Severity level (CRITICAL/WARNING/INFO)
   - Rule that triggered
   - Specific message

4. **Recommendations**: Prioritized action items
   - CRITICAL: Address immediately
   - HIGH: Address within 24 hours
   - MEDIUM: Address within 1 week

**Why structured reports matter (Chapter 8 best practice):**
- Executives need summaries, not raw data
- Consistent format enables trend tracking over time
- Recommendations drive action, not just awareness

In [0]:
# =============================================================================
# GENERATE REVIEW REPORT
# =============================================================================

print("📋 Generating Review Report...")
print("=" * 60)
print(f"PERIODIC REVIEW REPORT")
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

print(f"\n📊 EXECUTIVE SUMMARY")
print(f"   Review Period: Last 7 days")
print(f"   Total Requests Analyzed: {len(traffic_analysis_df)}")
print(f"   Overall Health: {'✅ HEALTHY' if len(review_report['alerts']) == 0 else '⚠️ NEEDS ATTENTION'}")

print(f"\n📈 KEY METRICS")
for metric, data in review_report['baseline_comparison'].items():
    status_icon = "✅" if data['status'] == 'ok' else "⚠️" if data['status'] == 'warning' else "🔴"
    print(f"   {status_icon} {metric}: {data['current']:.2f} (baseline: {data['baseline']:.2f}, change: {data['change_pct']:+.1f}%)")

print(f"\n🚨 ALERTS ({len(review_report['alerts'])})")
if review_report['alerts']:
    for alert in review_report['alerts']:
        print(f"   [{alert['severity'].upper()}] {alert['rule_name']}: {alert['message']}")
else:
    print("   No active alerts")

print(f"\n💡 RECOMMENDATIONS ({len(review_report['recommendations'])})")
if review_report['recommendations']:
    for i, rec in enumerate(review_report['recommendations'], 1):
        print(f"   {i}. [{rec['priority'].upper()}] {rec['area']}: {rec['action']}")
else:
    print("   No recommendations - system performing optimally")

print("\n" + "=" * 60)
print("END OF REPORT")
print("=" * 60)

### Step 8.3: Create Evaluation Dataset Versioning

**What we're doing:** Saving a timestamped version of our evaluation dataset so we can reproduce results and track changes over time.

**How versioning works:**

1. **Generate version string** from timestamp:
   ```python
   version = datetime.now().strftime("%Y%m%d_%H%M%S")
   # Result: "20241130_143022"
   ```

2. **Save versioned Delta table**:
   ```python
   versioned_table_name = f"{CATALOG}.{SCHEMA}.evaluation_dataset_v{version}"
   spark_df.write.saveAsTable(versioned_table_name)
   # Creates: main.llm_monitoring_lab.evaluation_dataset_v20241130_143022
   ```

3. **Log to MLflow** for discoverability:
   ```python
   mlflow.log_param("dataset_version", version)
   mlflow.log_param("dataset_table", versioned_table_name)
   mlflow.log_artifact("eval_dataset.csv")  # Downloadable copy
   ```

**Why version datasets?**
- **Reproducibility**: "What data did we use for the March evaluation?"
- **Debugging**: "Did the dataset change, or did the model regress?"
- **Compliance**: Audit trails for regulated industries
- **A/B testing**: Compare model performance on identical datasets

**Best practice:** Always version both the dataset AND the evaluation results together. Link them via MLflow run IDs.

In [0]:
# =============================================================================
# EVALUATION DATASET VERSIONING
# =============================================================================

# Save versioned evaluation dataset
version = datetime.now().strftime("%Y%m%d_%H%M%S")
versioned_table_name = f"{CATALOG}.{SCHEMA}.evaluation_dataset_v{version}"

# Save to Delta table with version
eval_spark_df = spark.createDataFrame(eval_df)
eval_spark_df.write.mode("overwrite").saveAsTable(versioned_table_name)

print(f"✅ Evaluation dataset versioned: {versioned_table_name}")

# Log version to MLflow
with mlflow.start_run(run_name=f"dataset_version_{version}"):
    mlflow.log_param("dataset_version", version)
    mlflow.log_param("dataset_table", versioned_table_name)
    mlflow.log_param("num_samples", len(eval_df))
    mlflow.log_param("categories", list(eval_df['category'].unique()))

    # Log dataset as artifact
    eval_df.to_csv(f"/tmp/eval_dataset_{version}.csv", index=False)
    mlflow.log_artifact(f"/tmp/eval_dataset_{version}.csv")

    print(f"   Logged to MLflow experiment: {EXPERIMENT_NAME}")

---

# Lab Summary and Key Takeaways

## What You Accomplished

In this hands-on lab, you successfully achieved all 7 learning objectives:

### 1. ✅ Designed an End-to-End Evaluation and Monitoring Workflow
- Set up MLflow experiment tracking for a production LLM system
- Created synthetic evaluation datasets with ground truth
- Established baseline metrics for ongoing comparison
- Implemented periodic review cycles

### 2. ✅ Compared Multiple Configurations and Justified Deployment Choice
- Ran evaluations on baseline and optimized configurations
- Applied the deployment decision framework with weighted criteria
- Made data-driven deployment recommendations with justification
- Logged decisions to MLflow for audit trail

### 3. ✅ Analyzed Inference Tables for Latency, Tokens, and RAG Grounding
- Queried inference tables to analyze production traffic patterns
- Identified slow queries and latency anomalies
- Performed RAG grounding analysis to detect hallucination signals
- Correlated context quality with answer quality

### 4. ✅ Distinguished Retrieval Failures from Model/Prompt Failures
- Implemented failure classification framework
- Diagnosed root causes: retrieval, model, or prompt issues
- Generated actionable recommendations for each failure type
- Used inference logs to identify patterns

### 5. ✅ Used Agent Monitoring for Multi-Step Workflows
- Implemented tracing for Q&A pipeline steps (preprocess, retrieve, generate, postprocess)
- Identified bottlenecks in the workflow
- Measured step-by-step latency contributions
- Analyzed trace data to optimize performance

### 6. ✅ Applied Cost Controls and Validated Without Quality Degradation
- Analyzed token usage patterns and cost drivers
- Implemented prompt compression and context truncation
- Validated that optimizations maintained quality thresholds
- Calculated projected cost savings

### 7. ✅ Selected Appropriate Evaluation Judges and Ground Truth Requirements
- Learned when to use LLM-as-judge vs ground truth comparison
- Understood trade-offs between evaluation approaches
- Applied the right evaluation strategy for RAG Q&A systems
- Implemented custom scorers using MLflow's new GenAI API

---

## Key Metrics Summary

In [0]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================

print("=" * 60)
print("           LAB COMPLETION SUMMARY")
print("=" * 60)

print(f"\n📊 Evaluation Metrics:")
print(f"   Samples Evaluated: {len(eval_df)}")
print(f"   Categories: {list(eval_df['category'].unique())}")

print(f"\n⚡ Performance Baseline:")
print(f"   P50 Latency: {current_metrics['latency_p50']:.2f} ms")
print(f"   P95 Latency: {current_metrics['latency_p95']:.2f} ms")
print(f"   Avg Tokens: {current_metrics['avg_total_tokens']:.1f}")

print(f"\n💰 Cost Optimization:")
if len(comparison_df) > 0:
    print(f"   Token Reduction: {comparison_df['token_reduction'].mean():.1f} tokens/query")
    print(f"   Projected Monthly Savings: ${savings/len(comparison_df)*10000*30:.2f}")

print(f"\n🔍 Anomaly Detection:")
print(f"   Latency Anomalies Detected: {len(anomalies['latency_anomalies'])}")
print(f"   Token Anomalies Detected: {len(anomalies['token_anomalies'])}")

print(f"\n📋 Review Status: {review_report['status']}")

print(f"\n✅ Lab completed successfully!")
print("=" * 60)

---

## Next Steps

After completing this lab, consider:

1. **Deploy to Production**: Apply these monitoring patterns to your production LLM systems
2. **Customize Metrics**: Add domain-specific evaluation metrics for your use case
3. **Automate Reviews**: Schedule periodic review workflows using Databricks Jobs
4. **Expand Alerting**: Integrate alerts with your team's notification systems (Slack, PagerDuty)
5. **Continuous Improvement**: Use evaluation results to guide model fine-tuning or prompt engineering

---

## Resources

- [Databricks MLflow Documentation](https://docs.databricks.com/mlflow/index.html)
- [Model Serving Best Practices](https://docs.databricks.com/machine-learning/model-serving/index.html)
- [LLM Evaluation with MLflow](https://mlflow.org/docs/latest/llms/llm-evaluate/index.html)
- [Databricks Monitoring](https://docs.databricks.com/lakehouse-monitoring/index.html)

---

**Congratulations on completing the Hands-On Lab!** 🎉

---

## Cleanup (Optional)

Run the following cell to clean up resources created during this lab.

In [0]:
# =============================================================================
# CLEANUP RESOURCES
# =============================================================================
# Uncomment and run to delete resources created during this lab

# Delete Model Serving endpoint
# w.serving_endpoints.delete(name=ENDPOINT_NAME)
# print(f"✅ Deleted endpoint: {ENDPOINT_NAME}")

# Delete Delta tables
# spark.sql(f"DROP SCHEMA IF EXISTS {CATALOG}.{SCHEMA} CASCADE")
# print(f"✅ Deleted schema: {CATALOG}.{SCHEMA}")

# Delete MLflow experiment
# Note: MLflow experiments cannot be permanently deleted, only archived
# mlflow.delete_experiment(experiment_id)
# print(f"✅ Archived experiment: {EXPERIMENT_NAME}")

print("⚠️ Cleanup code is commented out. Uncomment to delete resources.")